<center> <img src = "img\logo.png" alt="drawing" style="width:400px;">

<center>  

<span style="background-size: 600px;background:White;color:REd;font-size: 60px;font-family: Comic Sans MS">Кредитный скоринг Альфа банка</span>

# <span style="color:DeepSkyBlue">Задача</span>

**Задача**

Кредитный скоринг – важнейшая банковская задача. Стандартным подходом к ее решению   
является построение классических моделей машинного обучения, таких как логистическая   
регрессия и градиентный бустинг, на табличных данных, в том числе используя агрегации  
от каких-нибудь последовательных данных, например, транзакционных историй клиентов.   
Альтернативный подход заключается в использовании последовательных данных “как есть”,   
подавая их на вход рекуррентной нейронной сети.

В этом соревновании участникам предлагается решить задачу кредитного скоринга клиентов   
Альфа-Банка, используя только данные кредитных историй. [Источник](https://www.kaggle.com/competitions/alfa-bank-pd-credit-history)

**Данные**

Датасет соревнования устроен таким образом, что кредиты для тренировочной выборки взяты   
за период в М месяцев, а кредиты для тестовой выборки взяты за последующие K месяцев.

Каждая запись кредитной истории содержит самую разнообразную информацию о прошлом кредите   
клиента, например, сумму, отношение клиента к кредиту, дату открытия и закрытия, информацию   
о просрочках по платежам и др. Все публикуемые данные тщательно анонимизированы.

Целевая переменная – бинарная величина, принимающая значения 0 и 1, где 1 соответствует   
дефолту клиента по кредиту.


**Проверка решений**

Метрика соревнования – ROC AUC. Подробнее про метрику можно почитать, например, [здесь](https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/).

# <span style="color:DeepSkyBlue">Используемые библиотеки</span>

In [2]:
import os

# работа с регулярными выражениями
import re

# библиотеки для работы с табличными данными
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import fastparquet as fp

# генерация случайных чисел
import random
from random import randint
from sklearn.utils import shuffle

# библиотеки для построения графики
import seaborn as sns
import matplotlib.pyplot as plt #для визуализации
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots
import nbformat

# библиотеки для математических преобразований с массивами данных
import numpy as np
import mlx.core as mx
from sklearn import model_selection
from sklearn.model_selection import train_test_split

# библиотеки для работы с функциями(частичная передача аргументов в функцию)
from functools import partial

# библиотеки для работы со статистическими характеристиками
from scipy import stats
import statistics
from collections import Counter

# библиотеки для работы с pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

# проверка временного ряда на статичность
from statsmodels.tsa.stattools import adfuller

# Импортируем DBSCAN-кластеризацию
from sklearn.cluster import DBSCAN

# вставить картинку в Jupiter Notebook
from IPython.display import Image

# линейные модели машинного обучения
from sklearn import linear_model

# ансамбли моделей машинного обучения
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

# поиск гиперпараметров модели
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFE
import optuna

 # метрики
from sklearn import metrics

# библиотека для стандартизации данных
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

# сохранить полученные модели
import joblib
from joblib import dump, load

# сборщик мусора
import gc

# для ограничения времени выполнения функции
import signal
import func_timeout

# для отслеживания времени выполнения функции
import time

# очистить output
from IPython.display import clear_output

## <span style="color:DodgerBlue">Разработка инструментов преобразования данных</span>

### <span style="color:RoyalBlue">функция torow_transformer</span>

In [3]:
# функция torow_transformer

# Назначение: Преобразование признака столбца в признаки строки
#               с сохранением обратной последовательности в признаке.
#               (извлечение последних операций клиента)

# Внешние переменные функции: DataFrame, n_last
#   DataFrame - исоходый DataFrame
#   n_last - необходимое счисло послдених операций клиента
#   Структура DataFrame:
#       1. id
#       2. feature1
#       3. feature2
#       4. feature3
#       ...

# Результат работы функции: New DataFrame
#   Признаки New DataFrame:
#       1. id
#       2. feature1.1
#       3. feature1.2
#       4. feature1.3
#       ...
#       5. feature1.N
#       6. feature2.1
#       7. feature2.2
#       ...
#       8. feature2.N
#       ...
#   где featureX.1 - соотвествует последней операции клиента,
#       featureX.2 - соотвествует предпоследней операции клиента,
#       .....

# 2. dict_features - словарь (карта) признаков,
#    в котором отображается сокращения признаков и их расщифровка

# Алгоритм работы функции:
# 1. извлекаем признаки из данных DataFrame
# 3. формируем карту признаков dict_features:
#    3.1. каждый признак кодируется следующим образом: 'fn',
#         где n - порядковый номер признака
#    3.2. полные имена признаков задаются следующим образом: 'feature_N',
#         где N - порядок клиентской операции (большему N соотвествует, более ранняя операция)

# 4. Преобразуем данные к массиву
# 5. Группируем массив для каждого клиента
# 6. К групированному массиву прменяем следующие преобразования:
#    6.1. Обращаем порядок клиенских операций
#    6.2. выбираем посление n_last операций
#    6.3. Если число клиенски операций меньше чем n_last, дополняем их нулями
# 7. Преобразуем полученные данные к DataFrame

# Описание локальных переменных функции:
# 1. pd_data - исходный DataFrame
# 2. n_last - необходимое число последних операций клиента
# 3. list_id - список для хранения id клиентов внутри функции
# 4. list_features - список для хранения признаков исхдного dataframe внутри функции
# 5. dict_features - локальная карта признаков
# 6. rn_id - список количества операций для каждого клиента
# 6. array_data - данные преобразоанные к numpy-массиву
# 6. split_array - сгрупированные по клиентам данные преобразованные

# обьявлем функцию
def torow_func(dict_params):
    pd_data = dict_params['data']
    n_last = dict_params['n_last']

    # извлекаем список "id" клиентов
    list_id = pd_data['id'].unique().tolist()
    
    # извлекаем список признаков из данных        
    list_features = pd_data.columns.drop(['id','rn'])

    # формируем словарь для зашифрованных признаков
    dict_features = {}
    
    # заполним словарь dict_features
    num_f=0
    for feature in list_features:
        # шифруем признак: fk = "feature_agg_function"
        for num_feature in range(1,n_last+1):
            dict_features['f'+str(num_feature+num_f)] = feature+'_'+str(num_feature)
        num_f+=n_last
    

    # формируем словарь rn_id
    rn_id = pd_data.groupby('id')['id'].count().to_list()

    # для улучшения производительности преобразуем DataFrame в array-массив
    array_data = np.array(pd_data.iloc[:,2:]).transpose()

    # "порежем массив" по длине кредитной истории клиента
    split_array = np.array_split(array_data, np.cumsum(rn_id),axis=1)
    
    # определим порядок последующих преобразований в функции
    def transform_array(array_id):
        # обратим порядок клиентских операций 
        reverse_array_id = array_id[::,::-1]
        # выбрем после n операций клиента
        list_n_last = reverse_array_id[::,:n_last]
        # если клиенских операций было меньше чем n_last
        # дополним недастающие нулями и преобразуем данные к строке
        if len(list_n_last[0])<n_last:
            full_list_n_last = np.hstack((list_n_last,np.zeros((list_n_last.shape[0],n_last-len(list_n_last[0])),dtype='int64')))
            # преобразуем список к строке
            full_list_n_last = full_list_n_last.reshape(-1)
        else:
            full_list_n_last = list_n_last.reshape(-1)
        return full_list_n_last

    # применим transform_array преобразование к списку split_array
    list_data = np.array(list(map(transform_array,split_array)))[:-1]
    
    # преобразуем полученные данные к dataframe
    dataframe = pd.DataFrame(data=list_data, columns=dict_features.keys())

    # добавим столбец id
    dataframe.insert(0,'id',list_id)
    
    return dataframe, dict_features,rn_id


# преобразуем функции в инструмент для преобразования данных (Transformer)
torow_transformer = FunctionTransformer(torow_func)

In [4]:
# функция features_from_transform_data_torow

# Назначение: Извлечение из данных, над которомы совершено 
#             row_fich_transformer() преобразование, признаков  
#             соотвествующих заданному числу последних  
#             опреаций клиента n_last

# Внешние переменные функции: DataFrame
#   Признаки DataFrame:
#       1. n_last - необходимое число последних операций клиента
#       2. n_groups - число групп признаков в transform_data_torow
#       3. N_last - число последних операций клиента показанных в transform_data_torow

# Результат работы функции: 
# 1. list_n_last_features - список признаков в transform_data_torow
#    соотвествующий заданному числу n_last. 
    

# обьявлем функцию
def features_from_transform_data_torow(n_last,n_groups,N_last):
    # создадим список под необходимые признаки
    list_n_last_features = []
    
    # обьявим начальное значение в группе признаков
    n_start = 0
    
    for i in range(n_groups):
        for n in range(n_last):
            list_n_last_features.append(n+n_start)
        n_start+=N_last
    
    return list_n_last_features   

### <span style="color:RoyalBlue">функция diff_feature</span>

In [5]:
# функция diff_feature

# Назначение: Определение дифференциальных характеристик ряда 

# Внешние переменные функции: 
#           1.Series/np.array/list


# Результат работы функции: 
# 1. diff_list - Список из значений:
#                   1.1. speed - скорость изменения ряда;
#                   1.2. accel - ускорение изменения ряда;
#                   1.3. bias - смещение ряда;
#                   1.4. pulse - импульс ряда;

# обьявлем функцию
def diff_feature(data):
    # преобразуем данные к numpy массиву
    data = np.array(data)
    # расчитаем необходимые характеристики
    speed = round(float(np.diff(data,1).mean()),2)
    accel = round(float(np.diff(data,2).mean()),2)
    bias = round(float(np.diff(data,1).sum()),2)
    pulse = round(float(np.diff(data,2).sum()),2)
    # сформируем из найденных значений в список
    diff_list = [speed,accel,bias,pulse]
    return diff_list

### <span style="color:RoyalBlue">функция statistic_features</span>

In [6]:
# функция statistic_features

# Назначение: Извлечение основных статистических характеристик
#             из признаков в исходном DataFrame.

# Внешние переменные функции: DataFrame
#   Признаки DataFrame:
#       1. id
#       2. feature1
#       3. feature2
#       4. feature3
#       ...

# Результат работы функции: 
# 1. dataframe - таблица с данными. 
#    Признаки dataframe:
#       1. id
#       2. feature1_mean
#       3. fearture1_hmean
#       4. feature1_std
#       5. feature1_min
#       6. feature1_25%
#       7. feature1_50%
#       8. feature1_75%
#       9. feature1_max
#       10. feature1_mode
#       11. feature1_frequency_mode
#       12. feature2_mean
#       ...
    
# 2. dict_features - словарь (карта) признаков,
#    в котором отображается сокращения признаков и их расщифровка

# Алгоритм работы функции:
# 1. извлекаем признаки из данных
# 2. формируем карту признаков:
#    2.1. каждый признак кодируется следующим образом: 'fn' где n - порядковый номер признака
#    2.2. полные имена признаков задаются следующим образом:
#         2.2.1 если в исходном dataframe признак бинарный, то: "Исходное имя признака"+"binary"
#         2.2.2 если в исходном dataframe признак не бинарный, то: "Исходное имя признака"+"Статистическая характеристика"
# 3. для каждого клиента по каждому признаку из исходного dataframe расчитываем статистические характеристики
# 4. записываем полученные значение в новый dataframe

# Описание локальных переменных функции:
# 1. dict_agg_function - словарь из агригирующих функций
#       keys: имена для обращения к функциям:
#       values: lamda-функция, соотвествующей статистической характристики
# 2. list_features - список для хранения признаков исхдного dataframe внутри функции
# 3. list_id - список для хранения id клиентов внутри функции
# 4. dict_features - локальная карта признаков
# 5. k - номер признака в dict_features на текущей итерации
# 6. dataframe - результирующий dataframe

# обьявлем функцию
def statistic_features(pd_data):
    # формируем список из функций для статистических преобразований
    # предусмотрим работу функций на случай, если в массиве данных всего 1 строка
    dict_agg_function = {
    'ptp' : lambda x: 0 if len(x) <= 3 else np.ptp(x),
    'mean': lambda x: 0 if len(x) <= 3 else x.mean(), 
    'gmean' : lambda x: stats.gmean(x),   
    'hmean': lambda x: stats.gmean(x),
    'pmean25': lambda x: stats.pmean(x,25),
    'pmean50': lambda x: stats.pmean(x,50),
    'pmean75': lambda x: stats.pmean(x,75),
    'expectile25': lambda x: stats.expectile(x,0.25),
    'expectile50': lambda x: stats.expectile(x),
    'expectile75': lambda x: stats.expectile(x,0.75),
    'moment': lambda x: stats.moment(x),
    'std': lambda x: 0 if len(x) <= 3 else np.std(x),
    'min': lambda x: min(x),
    '20%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=20),
    '30%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=30),
    '40%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=40),
    '50%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=50),
    '60%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=60),
    '70%': lambda x: x.mean() if len(x) <= 3 else np.percentile(x,q=70),
    'max': lambda x: max(x),
    'mode': lambda x: statistics.mean(statistics.multimode(x)),
    'frequency_mode': lambda x: round(list(x).count(statistics.multimode(x)[0])*len(statistics.multimode(x))/len(x),2),
    'cov' : lambda x: 0 if len(x) <= 3 else np.cov(x),
    'histogram' : lambda x: 0 if len(x) <= 3 else np.histogram(x)[1].mean(), 
    'speed': lambda x: 0 if len(x) <= 3 else diff_feature(x)[0],
    'accel': lambda x: 0 if len(x) <= 3 else diff_feature(x)[1],
    'bias': lambda x: 0 if len(x) <= 3 else diff_feature(x)[2],
    'pulse': lambda x: 0 if len(x) <= 3  else diff_feature(x)[3]
    } 

    # напишем функцию для преобразования массива до статистических характеристик
    def stat_func(array_data): 
        # сформируем лист под результаты преобразования
        list_for_result = []
        # запишем все статистические харкетристики из словаря dict_agg_function
        for func in dict_agg_function.values():
            list_for_result.append(func(array_data))
        return np.array(list_for_result).round(3)

    # напишем функцию для применения функции stat_func к списку
    def submap(list_data):
        # расчитаем количество операция клиента
        max_rn = len(list_data[0])
        # получим статистические характристики массива
        list_stat_features = np.array(list(map(stat_func,list_data))).reshape(-1)
        return np.hstack((max_rn,list_stat_features))
    
    # извлекаем список "id" клиентов
    list_id = pd_data['id'].unique().tolist()

    # извлекаем список признаков из данных        
    list_features = pd_data.columns.drop(['id','rn'])
    
    # формируем словарь для зашифрованных признаков
    dict_features = {'f1':'count'}
    k=1 # порядковый номер защифрованного признака

    # заполним словарь dict_features
    for feature in list_features:
        # шифруем признак: fk = "feature_agg_function"
        for key_function in dict_agg_function.keys():
            k+=1
            dict_features['f'+str(k)] = feature+'_'+key_function

    # формируем список rn_id
    rn_id = pd_data.groupby('id')['id'].count().to_list()

    # для улучшения производительности преобразуем DataFrame в array-массив
    array_data = np.array(pd_data.iloc[:,2:]).transpose()

    # "порежем массив" по длине кредитной истории клиента
    split_array = np.array_split(array_data, np.cumsum(rn_id),axis=1)[:-1]

    # получем статические характеристики признаков
    stat_features = np.array(list(map(submap,split_array)))
    
    # Сформируем dataframe из полученных данных
    dataframe = pd.DataFrame(data=stat_features, columns=dict_features.keys())

    # добавим столбец id
    dataframe.insert(0,'id',list_id)

    return dataframe, dict_features

# преобразуем функции в инструмент для преобразования данных (Transformer)
stat_transformer = FunctionTransformer(statistic_features)

### <span style="color:RoyalBlue">функция corr_transform_to_force</span>

In [7]:
# функция corr_transform_to_force

# Назначение: из матрицы взаимных корреляций
#             выделить не корелирующие признаки

# Внешние переменные функции: 
#           1. df.corr() - матрица корреляций
#           2. threshold - порог значимости корреляции:
#               значение коэффициента корреляции, больше которого
#               признаки считаются скоррелированными.

# Пояснение: 
# Под силой корреляции будем понимать следующее: если коэффициент 
# коррелиции между признаками больше значения threshold, то принимаем,
# что между признаками сильная корреляционная связь значение коэффициента 
# коррелияции заменяем на 1, иначе корреляционная связь слабая и значение 
# коээфициента корреляции заменяем на 0

# Результат работы функции: 
# 1. corr_matrix - матрица корреляций(отражает силу корреляции)
# 2. list_ncorr_features - список не скореллированных признаков
# 3. corr_force - сила корреляции всей матрицы: отношение числа скоррелированных 
# признаков к числу всех признаков в матрице

# Описание локальных переменных функции:
# 1. coor_force - функция преобазующая значение
#        коэффициента корряляции в силу корреляции
# 2. corr_matrix - матрица отражающая силу корряляции между признаками
# 3. max_corr - максимальное число взаимных корреляций между признаками
# 4. list_ncorr_features - список не коррелируемых признаков


# обьявлем функцию
def corr_transform_to_force(matrix,threshold=0.7):
    list_features = matrix.index.tolist()
    
    
    # создадим функцию для разметки матрицы корреляции
    # 1 - корреляция признаков выше порога значимости threshold
    # 0 - корреляция признаков ниже порога значимости threshold
    corr_force = lambda x: 1 if x >threshold else 0
    # выполним разметку матрицы корреляции
    corr_matrix = matrix.map(lambda x: corr_force(x))
    
    # алгоритм отбора не коррелиарных признаков:
    #   1. Найдем признак с наибольшим числом взаимных корреляций
    #   2. удалим найденный признак
    #   3. составим матрицу корреляций из отсавшися признаков
    #   4. повторяем пункты 1-3 до тех пор пока в матрице не останутся 
    #       не коррелированные признаки

    # ищем наибольшее число взаимных корреляций среди признаков
    max_corr = corr_matrix.sum().max()

    while max_corr > 1:
        # определяем признак с наибольшим числом взаимных корреляций
        max_corr_feature = corr_matrix.sum()[corr_matrix.sum()==corr_matrix.sum().max()].index[0]
        # удалем признак из матрицы корреляций
        corr_matrix = corr_matrix.drop(max_corr_feature).drop(max_corr_feature,axis=1)
        max_corr = corr_matrix.sum().max()
    # запишем не скоррелированные признаки в список
    list_ncorr_features = corr_matrix.index.tolist()
    # найдем силу корреляции всей матрицы как отношение
    # количества скоррелированных признаков к всмеу количеству признаков
    corr_force = round(1-len(list_ncorr_features)/len(list_features),3)
    return corr_matrix, list_ncorr_features, corr_force

### <span style="color:RoyalBlue">функция search_DBSCAN_parameters</span>

In [8]:
# функция search_DBSCAN_parameters

# Назначение: Для подбора eps и min_samples параметров,
#               функция "прогоняет" DBSCAN кластеризацию 
#               с параметрами eps и min_samples
#               примающими значения из заданного диапазона.

# Внешние переменные функции: 
#           1. data - dataframe для кластеризации
#           2. r1 - начало диапазона
#           3. r2 - конец диапазона  
#           4. n - предпалагамое число кластеров      

# Результат работы функции: 
# 1. data_cluster - кластеризация данных при различных 
#       значениях параметров eps и min_samples

# Описание локальных переменных функции:
# 1. parametr_range - диапазон изменения параметров
# 2. dataframe_columns - колонки в результирующем dataframe
# 3. data_cluster - результрующий dataframe
# 4. index_cluster - текущая позиция в data_cluster
# 5. clustering - кластеризатор
# 6. list_cluster_values - список для заполнения текущими 
#                          значениями data_cluster

# обьявлем функцию
def search_DBSCAN_parameters(dataframe,r1,r2,n=3):
    # задаем диапозон измениния параметров
    parameter_range = range(r1,r2)
    # формируем заготовку для результирующего dataframe
    dataframe_columns = ['eps','min_samples',-1,0,1]
    # проверим что задано не меньше минимального количества кластеров
    if n<=3: 
        data_cluster = pd.DataFrame(columns=dataframe_columns)
    else: 
        for claster in range(4,n+1):
            dataframe_columns.append(claster-2)
        data_cluster = pd.DataFrame(columns=dataframe_columns)
    # задаем начально значение индекса в data_cluster
    index_cluster = 0

    # для подсчета обьектов в кластерах создадим dataframe
    dataframe_count = pd.DataFrame()
    
    # "прогоняем" DBSCAN кластеризациию по диапазону параметров
    for eps in parameter_range:
        
        for min_samples in parameter_range:
            print('current eps:',eps,'  current min_samples:', min_samples, end='\r')
            # запускаем кластеризацию с текущими параметрами
            clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(dataframe)
            # добавлем к данным столбец с разметкой
            dataframe_count['clater'] = clustering.labels_
            # формируем пустой список для заполнения
            list_cluster_values = []
            # добавлеям в список текущие параметры
            list_cluster_values.append(eps)
            list_cluster_values.append(min_samples)
            # добавлем в список количество обьктов в каждом кластере
            for column in dataframe_columns[2:]:
                list_cluster_values.append(len(dataframe_count['clater'][dataframe_count['clater']==column]))
                
            # заполняем dataframe  текущими данными
            data_cluster.loc[index_cluster] = list_cluster_values
            index_cluster +=1
            # сбрасываем dataframe_count
            dataframe_count = pd.DataFrame()
    return data_cluster

### <span style="color:RoyalBlue">функция generate_samples</span>

In [9]:
# функция generate_samples

# Назначение: для генерации индексов выборок данных

# Внешние переменные функции: 
#           1. max - определяет максимальное значение множества
#               из которого формируются выборки
#           2. n - количество выборок
#           3. k - мощность одной выборки
     

# Результат работы функции: 
# 1. samples_list - список с выбороками

# алгоритм работы:
# 1. задаем отрезок натурального ряда N мощностью max и добавлем в него 0.
#       In = N U {0}, I = {0,1,2,3,4,5,..,max}
# 2. если мощность множества In больше, необходимого количества элементов
#       cardo(In) > n x k , то из множества In формируем n случайных выборок
# размера k без повторения.
# 3. если мощность множества In меньше, необходимого количества элементов
#       cardo(In) < n x k , то из множества In формируем случайные выборки
# размера k без повторения, до тех пор пока не закончится множество In.
# После, добираем недостающее количество выборок случайными выборками 
# размера k из множества In с повторением (bootstrap метод).


# обьявлем функцию
def generate_samples(max,n,k,random_state = None):
    # создадим список под результат
    samples_list = []
    # формируем множество натуральных числе от 0 до max
    In = list(range(max+1))
    # Будем выполнять код пока не наберем необходимого количества выборок
    # нарушим порядок в множестве
    In = shuffle(In,random_state=random_state)
    # random.shuffle(In)

    # задаим границы извлечения данных из In
    In_start = 0
    In_end = k
    while len(samples_list) < n:
        # сформируем список под одну выборку
        sample = []
        # первые списки будем наполнять значениеми из множества In
        # без повторения, до тех пор пока все значения из множества In
        # не распределяться по выборкам
        if len(In)-In_end >= 0:
            sample.extend(In[In_start:In_end])
        else:                    
            # если элементов во множестве In недостаточно,
            # запоняем выборку "остатками" 
            sample.extend(In[In_start:])

            # остальные данные заполняем методом bootstrap
            # выполнем код пока не заполним выборку k значениями
            while len(sample) < k:
                # генерируем случайное число из диапазона от 0 до len(In)-1
                random_index = randint(0,len(In)-1)
                # добавляем значение из множества In с индексом random_index
                # в список index_list
                sample.append(In[random_index])

        # после того как мы набрали значения в выборку отправлем ее в samples_list
        samples_list.append(sample)
        # переходим к следующим данным в множестве In
        In_start+=k
        In_end+=k

    return samples_list

### <span style="color:RoyalBlue">функция my_train_test_split</span>

In [10]:
def my_train_test_split(X,y,random_state=42,train_size=0.8,):
    # если разбиение без стратификации

    # зададим число элементов в выборке train
    len_train = round(len(y)*train_size)
    # формируем множество натуральных чисел от 0 до max
    list_random_index = list(range(len(y)))
    # нарушим порядок в множестве
    list_random_index = shuffle(list_random_index,random_state=random_state)
    # формируем список индексов под train выборку
    train_samples = list_random_index[:len_train]
    # формируем список индексов под test выборку
    test_samples = list_random_index[len_train:]
    # выполнем код пока не заполним выборку k значениями
   
    X_train = X.iloc[train_samples]
    y_train = y.iloc[train_samples]
    X_test = X.iloc[test_samples]
    y_test = y.iloc[test_samples]

    return X_train, y_train, X_test, y_test

### <span style="color:RoyalBlue">функция class_1_percent_samples</span>

In [11]:
# функция class_1_percent_samples

# Назначение: для генерации индексов сбалансированных выборок

# Внешние переменные функции: 
#           1. data_target - массив из id и значений класса
#           2. class_1_percent - процент класса 1 в результирующей выборке
#           3. random_state - параметр для обеспечения воспроизваодимости функции
     

# Результат работы функции: 
# 1. samples_list - список со сблансированными выбороками

# обьявлем функцию
def class_1_percent_samples(data_target,class_1_percent,random_state = None):
    # приведем данные к нужной форме
    data_target = pd.DataFrame(data=np.array(data_target),columns =['id','flag'])
    
    # разделим клиентов  по признаку flag
    flag_0 = data_target[data_target['flag']==0].reset_index(drop=True)
    flag_1 = data_target[data_target['flag']==1].reset_index(drop=True)

    # определим класс большинства
    if flag_1.shape[0] > flag_0.shape[0]:
        majority_class = flag_1
        minority_class = flag_0
        # расчитаем необходимую величину выборки majority_class
        majority_class_size = round(minority_class.shape[0]*(class_1_percent)/(1-class_1_percent))
        # с помощью функции generate_samples сформируем выборку для majority_class
        samples_majority_class= generate_samples(majority_class.shape[0]-1,1,majority_class_size,random_state=random_state)
    else:
        majority_class = flag_0
        minority_class = flag_1
        # расчитаем необходимую величину выборки majority_class
        majority_class_size = round(minority_class.shape[0]*(1-class_1_percent)/(class_1_percent))
        # с помощью функции generate_samples сформируем выборку для majority_class
        samples_majority_class= generate_samples(majority_class.shape[0]-1,1,majority_class_size,random_state=random_state)
    
    # сформируем список выбороки с заданным процентом класс 1
    samples_list_id = minority_class['id'].values.tolist()+majority_class['id'].iloc[samples_majority_class[0]].tolist()

    return samples_list_id

# <span style="color:DeepSkyBlue">Процесс машинного обучения (ML-Machine Learning)</span>

Постановка задачи в рамках Machine Learning:
1. Для решения задачи построем блендинг моделей. 

2. В качестве базовых и метамоделей рассмотрим следующие классические модели классификации:
    - linear_model.LogisticRegression (Логистическая регрессия);
    - RandomForestClassifier (Деревья решений);
    - HistGradientBoostingClassifier (Градиентный бустинг).

3. В результате, преобразования данных было получено два пространства признаков (torow и stat признаки),  
состоящих из 6 подпространств:
    - date features;
    - late payments features; 
    - credit features;
    - relative features;
    - payments features;
    - service features.

4. На первом этапе построения потроения блендинга, сфокусируем обучение базовых моделей,   
на каждом подпространстве в отдельности друго от друга.  

5. На втором этапе построения блендинга, обучим несколько групп метамоделей.   
Первая группа метамоделей в качестве метапризнаков использует предсказания базовых моделей,    
обученных на пространстве признаков torow.  
Вторая группа метамоделей в качестве метапризнаков использует предсказания базовых моделей,    
обученных на пространстве признаков stat.

6. На третьем этапе построения блендинга метамодель обучится на метапризнаках пространства 
torow и stat.

<center> <img src = "img\Blending.jpg" alt="drawing" style="width:1400px;">

## <span style="color:DodgerBlue">Второй этап построения блендинга моделей (first metamodels) </span>

### <span style="color:RoyalBlue">Формирование данных для обучения first meta models</span>

Попробуем улучшить качество модели за счет блендинга.  
Сформируем блендинга из рассмотренных базовых модей:
- $Logistic$ $Regression$;
- $Hist$ $Gradient$ $Boosting$ $Classifier$,  
обученных на данных $transform$ $data$ $torow$ и $transform$ $data$ $stat$.

#### <span style="color:MediumBlue">Формирование метапризнаков из моделей обученных на transform data torow данных</span>

Проанализируем качество моделей обученных на torow данных

In [12]:
# определим пространство признаков
dict_spaces = {
    'date' : 8,
    'late': 12,
    'credit': 4,
    'relative' : 6,
    'payments': 25,
    'service': 4}

# сформируем список из базовых моделей
list_base_models = ['LogisticRegression','HistGradientBoostingClassifier']

# заготовка для общего списка метрик
metrics_torow_pd = pd.DataFrame()

In [13]:
for base_model in list_base_models:
    # сформируем общий список метрик моделей обученных на torow данных
    for feature_space in dict_spaces.keys():
        # загрузим обьект исcледования optuna
        optuna_study_torow = optuna.load_study(study_name=base_model+'_'+feature_space+'_torow',
                                storage='sqlite:///optuna_studies.db')
        
        # из полученного обьекта сформируем Data Frame
        optuna_study_torow_pd = optuna_study_torow.trials_dataframe()[['values_0','values_1','values_2','values_3','values_4']].dropna()

        # переименуем столбы
        optuna_study_torow_pd.rename(columns={
            'values_0': 'roc_train', 
            'values_1': 'roc_valid', 
            'values_2': 'f1_score', 
            'values_3': 'recall_1',
            'values_4': 'precision_1'
        },inplace=True)

        # добавим текущще подпростраство в общий список
        metrics_torow_pd = pd.concat([metrics_torow_pd,optuna_study_torow_pd],ignore_index=True)

# посмотрим на результат
metrics_torow_pd

,roc_train,roc_valid,f1_score,recall_1,precision_1
0,0.585156,0.580831,0.000000,0.000000,0.000000
1,0.575883,0.571258,0.067882,1.000000,0.035133
2,0.494216,0.488417,0.067875,1.000000,0.035130
3,0.588762,0.580936,0.000000,0.000000,0.000000
4,0.574669,0.568678,0.070770,0.901318,0.036831
...,...,...,...,...,...
4196,0.670899,0.636532,0.095426,0.106272,0.086589
4197,0.656207,0.633204,0.069222,0.990012,0.035865
4198,0.646931,0.636672,0.067917,0.999201,0.035153
4199,0.644888,0.634490,0.069238,0.991610,0.035871


In [14]:
# проанализируем метрики всех моделей обученных на torow данных
metrics_torow_pd.describe()

,roc_train,roc_valid,f1_score,recall_1,precision_1
count,4201.000000,4201.000000,4201.000000,4201.000000,4201.000000
mean,0.638668,0.619358,0.072161,0.428013,0.066453
std,0.050347,0.033692,0.034689,0.370498,0.058417
min,0.483961,0.485817,0.000000,0.000000,0.000000
25%,0.595874,0.597988,0.067865,0.062325,0.037762
50%,0.636853,0.623173,0.076725,0.346784,0.052280
75%,0.683712,0.635425,0.095658,0.799041,0.082126
max,0.747136,0.687650,0.141577,1.000000,1.000000


Для блендинга выберем модели c значениями метрик на валидационном наборе выше верхнего квартиля:
- модели с значением метрики $ROC AUC \geq 0.637$;
- модели с значением метрики $precision_1 \geq 0.081$;
- модели с значением метрики $f1_{score} \geq 0.1$;

In [15]:
# зададим минимальные значения метрик
min_roc_auc = float(metrics_torow_pd.describe()['roc_valid']['75%'])
min_precision = float(metrics_torow_pd.describe()['precision_1']['75%'])
min_f1 = float(metrics_torow_pd.describe()['f1_score']['75%'])

In [16]:
# сформируем заготовку под метаданные для обучения и проверики метамодели 
features_first_meta_torow = pd.read_csv('target/target_test.csv')

# для выбора sceler преобразвания понадобится словарь
dict_scalers ={
    'MinMaxScaler':MinMaxScaler(),
    'RobustScaler':RobustScaler(),
    'StandardScaler':StandardScaler(),
}

# задаим количество отбираемых "лучших" моделей
best_models_number = 15

##### <span style="color:MediumSlateBlue">Формирование метапризнаков от модели *Logistic Regression* обученной сбалансированными *transform data torow* данными</span>

In [17]:
# сформируем метапризнаки для каждого подпространства признаков
for feature_space in dict_spaces.keys():
    # обьявим количество признаков подпространства
    count_features = dict_spaces[feature_space]

    # загрузим обьект исcледования optuna
    optuna_study_lr_torow = optuna.load_study(study_name='LogisticRegression_'+feature_space+'_torow',
                               storage='sqlite:///optuna_studies.db')
    
    # из полученного обьекта сформируем Data Frame
    optuna_study_lr_torow_pd = optuna_study_lr_torow.trials_dataframe().dropna()

    # переименуем столбы
    optuna_study_lr_torow_pd.rename(columns={
        'values_0': 'roc_train', 
        'values_1': 'roc_valid', 
        'values_2': 'f1_score', 
        'values_3': 'recall_1',
        'values_4': 'precision_1'
    },inplace=True)

    # удалим ненужные столбцы
    optuna_study_lr_torow_pd.drop(['datetime_start','datetime_complete','state','duration'],axis=1, inplace=True)

    # выберем точки с лучшими значениями метрики roc auc на валидационном наборе
    best_roc_lr_torow = optuna_study_lr_torow_pd[optuna_study_lr_torow_pd['roc_valid']>=min_roc_auc].sort_values(by='roc_valid',ascending=False)[:best_models_number] 

    # выберем точки с лучшими значениями метрики precion класса 1 на валидационном наборе
    best_precion_lr_torow = optuna_study_lr_torow_pd[optuna_study_lr_torow_pd['precision_1']>=min_precision].sort_values(by='precision_1',ascending=False)[:best_models_number]   

    # выберем точки в которых максимальна метрика f1-score класса 1 (precision_1=recall_1)
    best_f1_lr_torow = optuna_study_lr_torow_pd[optuna_study_lr_torow_pd['f1_score']>=min_f1].sort_values(by='f1_score',ascending=False)[:best_models_number]

    # сформируем набор признаков для лучших моделей
    best_models_lr_torow = pd.concat([best_roc_lr_torow,best_precion_lr_torow,best_f1_lr_torow]).drop_duplicates().reset_index(drop=True)

    # сформируем список индексов best_models_lr_torow
    list_index = best_models_lr_torow.index

    # сформируем  метапризнаки для обучения метамодели
    # для этого необходиом обучить модель LogisticRegression с выбранными гиперпараметрами
    for index in list_index:

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Current space: ', feature_space)
        print('Number of indexes: ', max(list_index))
        print('Current index: ', index)
        
        # сформируем необходимые параметры
        C = best_models_lr_torow.loc[index,'params_C']
        solver = best_models_lr_torow.loc[index,'params_solver']
        class_weight = {0:best_models_lr_torow.loc[index,'params_class_0_weight'],
                        1:best_models_lr_torow.loc[index,'params_class_1_weight']}
        n_last = int(best_models_lr_torow.loc[index,'params_n_last'])
        scaler = best_models_lr_torow.loc[index,'params_scaler']
        class_1_percent = best_models_lr_torow.loc[index,'params_class_1_percent']
        random_state = int(best_models_lr_torow.loc[index,'params_random_state'])

        # с помощью функции features_from_transform_data_torow извлечем  
        # из данных transform_data_torow
        # признаки сооствествующие n_last последним клиенским операциям
        list_n_last_features = features_from_transform_data_torow(n_last,count_features,25)

        # сохраним list_n_last_features для дальнейшего воспроизведения
        dump(list_n_last_features, 'models/base/list_n_last_features/'+'list_n_last_features_LRTR_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # обьявим scaler
        scaler = dict_scalers[scaler]

        # загружаем обучующие наборы
        print('Loading train data')
        X_train_pd = fp.ParquetFile('features/base/torow/'+feature_space+'_torow_train').to_pandas()
        y_train_pd = pd.read_csv('target/target_train.csv')

        # поготовим данные y_train_pd к работе с функцией class_1_percent_samples
        y_train_pd.set_index('id',drop=False,inplace=True)

        # подготовим данные для обучения модели
        # с помощью функции class_1_percent_samples зададим долю 
        # класса 1
        list_c1_percent_id = class_1_percent_samples(y_train_pd,class_1_percent,random_state=random_state)[:1000000]

        # сформируем данные для обучения базовой модели
        X_train_s_balanced = scaler.fit_transform(np.array(X_train_pd.loc[list_c1_percent_id])[:,list_n_last_features])
        y_train_balanced = y_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

        # сохраним scaler для дальнейшего воспроизведения
        dump(scaler, 'models/base/scalers/'+'scaler_torow_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # освободим память от "тяжелых" и ненужных файлов
        del X_train_pd, y_train_pd
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('start train')
        # обучаем модель LogisticRegression с наилучшеми параметрами
        logistic_regression = linear_model.LogisticRegression(
                C=C,
                solver = solver,
                class_weight=class_weight,
                random_state=random_state,
                max_iter=10000)
        logistic_regression.fit(X_train_s_balanced,y_train_balanced)
        # для того чтобы следить за выполнением цикла введем индикатор
        print('finished train')

        # сохраним модель для дальнейшего воспроизведения
        dump(logistic_regression, 'models/base/'+'LRTR_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # удаляем крупные файлы чтобы высвободить память 
        del X_train_s_balanced,y_train_balanced
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Loading test data')
        # подгружаем данные для тестирования
        X_test = fp.ParquetFile('features/base/torow/'+feature_space+'_torow_test').to_pandas().to_numpy()[:,list_n_last_features]
        # сформируем данные для проверки модели
        X_test_s = scaler.transform(X_test)
        # для метрик ROC AUC делаем предсказание модели в виде вероятности и записываем в методанные
        features_first_meta_torow['LRTR_'+feature_space+'_'+str('0'+str(index))[-2:]] = logistic_regression.predict_proba(X_test_s)[:,1]
        # удаляем крупные файлы чтобы высвободить память 
        del X_test_s, X_test
        gc.collect()
        clear_output()

# сохраним полученные метапризнаки
fp.write('features/firstmeta/features_first_meta_torow',features_first_meta_torow)

##### <span style="color:MediumSlateBlue">Формирование метапризнаков от модели *Gradient Boosting Classifier* обученной сбалансированными *transform data torow* данными</span>

In [18]:
# сформируем метапризнаки для каждого подпространства признаков
for feature_space in dict_spaces.keys():
    # обьявим количество признаков подпространства
    count_features = dict_spaces[feature_space]

    # загрузим обьект исcледования optuna
    optuna_study_gb_torow = optuna.load_study(study_name='HistGradientBoostingClassifier_'+feature_space+'_torow',
                               storage='sqlite:///optuna_studies.db')
    
    # из полученного обьекта сформируем Data Frame
    optuna_study_gb_torow_pd = optuna_study_gb_torow.trials_dataframe().dropna()

    # переименуем столбы
    optuna_study_gb_torow_pd.rename(columns={
        'values_0': 'roc_train', 
        'values_1': 'roc_valid', 
        'values_2': 'f1_score', 
        'values_3': 'recall_1',
        'values_4': 'precision_1'
    },inplace=True)

    # удалим ненужные столбцы
    optuna_study_gb_torow_pd.drop(['datetime_start','datetime_complete','state','duration'],axis=1, inplace=True)

    # выберем точки с лучшими значениями метрики roc auc на валидационном наборе
    best_roc_gb_torow = optuna_study_gb_torow_pd[optuna_study_gb_torow_pd['roc_valid']>=min_roc_auc].sort_values(by='roc_valid',ascending=False)[:best_models_number] 

    # выберем точки с лучшими значениями метрики precion класса 1 на валидационном наборе
    best_precion_gb_torow = optuna_study_gb_torow_pd[optuna_study_gb_torow_pd['precision_1']>=min_precision].sort_values(by='precision_1',ascending=False)[:best_models_number]   

    # выберем точки в которых максимальна метрика f1-score класса 1 (precision_1=recall_1)
    best_f1_gb_torow = optuna_study_gb_torow_pd[optuna_study_gb_torow_pd['f1_score']>=min_f1].sort_values(by='f1_score',ascending=False)[:best_models_number]

    # сформируем набор признаков для лучших моделей
    best_models_gb_torow = pd.concat([best_roc_gb_torow,best_precion_gb_torow,best_f1_gb_torow]).drop_duplicates().reset_index(drop=True)

    # сформируем список индексов best_models_gb_torow
    list_index = best_models_gb_torow.index

    # сформируем  метапризнаки для обучения метамодели
    # для этого необходиом обучить модель HistGradientBoostingClassifier с выбранными гиперпараметрами
    for index in list_index:

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Current space: ', feature_space)
        print('Number of indexes: ', max(list_index))
        print('Current index: ', index)
        
        # сформируем необходимые параметры
        learning_rate = best_models_gb_torow.loc[index,'params_learning_rate']
        max_iter = best_models_gb_torow.loc[index,'params_max_iter']
        max_leaf_nodes = best_models_gb_torow.loc[index,'params_max_leaf_nodes']
        max_depth = best_models_gb_torow.loc[index,'params_max_depth']
        min_samples_leaf = best_models_gb_torow.loc[index,'params_min_samples_leaf']
        l2_regularization = best_models_gb_torow.loc[index,'params_l2_regularization']
        max_features = best_models_gb_torow.loc[index,'params_max_features']
        class_weight = {0:best_models_gb_torow.loc[index,'params_class_0_weight'],
                        1:best_models_gb_torow.loc[index,'params_class_1_weight']}
        n_last = best_models_gb_torow.loc[index,'params_n_last']
        class_1_percent = best_models_gb_torow.loc[index,'params_class_1_percent']
        random_state = best_models_gb_torow.loc[index,'params_random_state']

        # с помощью функции features_from_transform_data_torow извлечем  
        # из данных transform_data_torow
        # признаки сооствествующие n_last последним клиенским операциям
        list_n_last_features = features_from_transform_data_torow(n_last,count_features,25)

            # сохраним list_n_last_features для дальнейшего воспроизведения
        dump(list_n_last_features, 'models/base/list_n_last_features/'+'list_n_last_features_GBTR_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # загружаем обучующие наборы
        print('Loading train data')
        X_train_pd = fp.ParquetFile('features/base/torow/'+feature_space+'_torow_train').to_pandas()
        y_train_pd = pd.read_csv('target/target_train.csv')

        # поготовим данные y_train_pd к работе с функцией class_1_percent_samples
        y_train_pd.set_index('id',drop=False,inplace=True)

        # подготовим данные для обучения модели
        # с помощью функции class_1_percent_samples зададим долю класса 1
        list_c1_percent_id = class_1_percent_samples(y_train_pd,class_1_percent,random_state=random_state)[:1000000]

        # сформируем данные для обучения модели
        X_train_balanced = np.array(X_train_pd.loc[list_c1_percent_id])[:,list_n_last_features]
        y_train_balanced = y_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

        # освободим память от "тяжелых" и ненужных файлов
        del X_train_pd, y_train_pd
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('start train')
        # обучаем модель HistGradientBoostingClassifier с наилучшеми параметрами
        gradient_boosting = HistGradientBoostingClassifier(
                learning_rate = learning_rate,
                max_iter = max_iter,
                max_leaf_nodes = max_leaf_nodes,
                max_depth = max_depth,
                min_samples_leaf = min_samples_leaf,
                l2_regularization = l2_regularization,
                max_features = max_features,
                class_weight = class_weight,
                random_state=42)
        gradient_boosting.fit(X_train_balanced,y_train_balanced)
        # для того чтобы следить за выполнением цикла введем индикатор
        print('finished train')

        # сохраним модель для дальнейшего воспроизведения
        dump(gradient_boosting, 'models/base/'+'GBTR_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # удаляем крупные файлы чтобы высвободить память 
        del X_train_balanced,y_train_balanced
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Loading test data')
        # подгружаем данные для тестирования
        X_test = fp.ParquetFile('features/base/torow/'+feature_space+'_torow_test').to_pandas().to_numpy()[:,list_n_last_features]
        # для метрик ROC AUC делаем предсказание модели в виде вероятности и записываем в методанные
        features_first_meta_torow['GBTR_'+feature_space+'_'+str('0'+str(index))[-2:]] = gradient_boosting.predict_proba(X_test)[:,1]
        # удаляем крупные файлы чтобы высвободить память 
        del X_test
        gc.collect()
        clear_output()

# сохраним полученные метапризнаки
fp.write('features/firstmeta/features_first_meta_torow',features_first_meta_torow)

#### <span style="color:MediumBlue">Формирование метапризнаков из моделей обученных на transform data stat данных</span>

Проанализируем качество моделей обученных на stat данных

In [19]:
# определим пространство признаков
dict_spaces = {
    'date' : 8,
    'late': 12,
    'credit': 4,
    'relative' : 6,
    'payments': 25,
    'service': 4}

# сформируем список из базовых моделей
list_base_models = ['LogisticRegression','HistGradientBoostingClassifier']

# заготовка для общего списка метрик
metrics_stat_pd = pd.DataFrame()

In [20]:
for base_model in list_base_models:
    # сформируем общий список метрик моделей обученных на torow данных
    for feature_space in dict_spaces.keys():
        # загрузим обьект исcледования optuna
        optuna_study_stat = optuna.load_study(study_name=base_model+'_'+feature_space+'_stat',
                                storage='sqlite:///optuna_studies.db')
        
        # из полученного обьекта сформируем Data Frame
        optuna_study_stat_pd = optuna_study_stat.trials_dataframe()[['values_0','values_1','values_2','values_3','values_4']].dropna()

        # переименуем столбы
        optuna_study_stat_pd.rename(columns={
            'values_0': 'roc_train', 
            'values_1': 'roc_valid', 
            'values_2': 'f1_score', 
            'values_3': 'recall_1',
            'values_4': 'precision_1'
        },inplace=True)

        # добавим текущще подпростраство в общий список
        metrics_stat_pd = pd.concat([metrics_stat_pd,optuna_study_stat_pd],ignore_index=True)

# посмотрим на результат
metrics_stat_pd

,roc_train,roc_valid,f1_score,recall_1,precision_1
0,0.570751,0.566699,0.000000,0.000000,0.000000
1,0.586950,0.581612,0.085706,0.241710,0.052088
2,0.571209,0.569071,0.000000,0.000000,0.000000
3,0.581467,0.577797,0.000000,0.000000,0.000000
4,0.583944,0.577297,0.000000,0.000000,0.000000
...,...,...,...,...,...
3595,0.658287,0.641242,0.099141,0.536956,0.054612
3596,0.662682,0.641477,0.110810,0.351578,0.065770
3597,0.659675,0.641895,0.093336,0.091490,0.095258
3598,0.658903,0.640088,0.068801,0.989213,0.035640


In [21]:
# проанализируем метрики всех моделей обученных на torow данных
metrics_stat_pd.describe()

,roc_train,roc_valid,f1_score,recall_1,precision_1
count,3600.000000,3600.000000,3600.000000,3600.000000,3600.000000
mean,0.645177,0.630109,0.076166,0.270411,0.076349
std,0.039747,0.025895,0.038083,0.274393,0.058625
min,0.569406,0.566111,0.000000,0.000000,0.000000
25%,0.614292,0.616844,0.056656,0.041550,0.052477
50%,0.640839,0.626101,0.088602,0.181382,0.067204
75%,0.677664,0.657208,0.104749,0.414702,0.092143
max,0.739731,0.671994,0.134240,1.000000,1.000000


Для блендинга выберем модели c значениями метрик на валидационном наборе выше верхнего квартиля:
- модели с значением метрики $ROC AUC \geq 0.652$;
- модели с значением метрики $precision_1 \geq 0.092$;
- модели с значением метрики $f1_{score} \geq 0.1$;

In [22]:
# зададим минимальные значения метрик
min_roc_auc = float(metrics_stat_pd.describe()['roc_valid']['75%'])
min_precision = float(metrics_stat_pd.describe()['precision_1']['75%'])
min_f1 = float(metrics_stat_pd.describe()['f1_score']['75%'])

In [23]:
# сформируем заготовки под метаданные для обучения и проверики метамодели 
features_first_meta_stat = pd.read_csv('target/target_test.csv')

# для выбора scaler преобразвания понадобится словарь
dict_scalers ={
    'MinMaxScaler':MinMaxScaler(),
    'RobustScaler':RobustScaler(),
    'StandardScaler':StandardScaler(),
}

# задаим количество отбираемых "лучших" моделей
best_models_number = 15

##### <span style="color:MediumSlateBlue">Формирование метапризнаков от модели *Logistic Regression* обученной сбалансированными *transform data stat* данными</span>

In [24]:
# сформируем метапризнаки для каждого подпространства признаков
for feature_space in dict_spaces.keys():
    # обьявим количество признаков подпространства
    count_features = dict_spaces[feature_space]

    # для работы функции corr_transform_to_force
    # подгрузим DataFrame с матрицей корреляции признаков
    corr_matrix = fp.ParquetFile('features/base/stat/corr_matrix_'+feature_space).to_pandas()

    # загрузим обьект исcледования optuna
    optuna_study_lr_stat = optuna.load_study(study_name='LogisticRegression_'+feature_space+'_stat',
                               storage='sqlite:///optuna_studies.db')
    
    # из полученного обьекта сформируем Data Frame
    optuna_study_lr_stat_pd = optuna_study_lr_stat.trials_dataframe().dropna()

    # переименуем столбы
    optuna_study_lr_stat_pd.rename(columns={
        'values_0': 'roc_train', 
        'values_1': 'roc_valid', 
        'values_2': 'f1_score', 
        'values_3': 'recall_1',
        'values_4': 'precision_1'
    },inplace=True)

    # удалим ненужные столбцы
    optuna_study_lr_stat_pd.drop(['datetime_start','datetime_complete','state','duration'],axis=1, inplace=True)

    # выберем точки с лучшими значениями метрики roc auc на валидационном наборе
    best_roc_lr_stat = optuna_study_lr_stat_pd[optuna_study_lr_stat_pd['roc_valid']>=min_roc_auc].sort_values(by='roc_valid',ascending=False)[:best_models_number] 

    # выберем точки с лучшими значениями метрики precion класса 1 на валидационном наборе
    best_precion_lr_stat = optuna_study_lr_stat_pd[optuna_study_lr_stat_pd['precision_1']>=min_precision].sort_values(by='precision_1',ascending=False)[:best_models_number]   

    # выберем точки в которых максимальна метрика f1-score класса 1 (precision_1=recall_1)
    best_f1_lr_stat = optuna_study_lr_stat_pd[optuna_study_lr_stat_pd['f1_score']>=min_f1].sort_values(by='f1_score',ascending=False)[:best_models_number]

    # сформируем набор признаков для лучших моделей
    best_models_lr_stat = pd.concat([best_roc_lr_stat,best_precion_lr_stat,best_f1_lr_stat]).drop_duplicates().reset_index(drop=True)

    # сформируем список индексов best_models_lr_stat
    list_index = best_models_lr_stat.index

    # сформируем  метапризнаки для обучения метамодели
    # для этого необходиом обучить модель LogisticRegression с выбранными гиперпараметрами
    for index in list_index:

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Current space: ', feature_space)
        print('Number of indexes: ', max(list_index))
        print('Current index: ', index)
        
        # сформируем необходимые параметры
        C = best_models_lr_stat.loc[index,'params_C']
        solver = best_models_lr_stat.loc[index,'params_solver']
        class_weight = {0:best_models_lr_stat.loc[index,'params_class_0_weight'],
                        1:best_models_lr_stat.loc[index,'params_class_1_weight']}
        threshold = best_models_lr_stat.loc[index,'params_threshold']
        scaler = best_models_lr_stat.loc[index,'params_scaler']
        class_1_percent = best_models_lr_stat.loc[index,'params_class_1_percent']
        random_state = int(best_models_lr_stat.loc[index,'params_random_state'])

        # с помощью функции corr_transform_to_force получим
        # список не коррелируемых признаков
        list_ncorr_features = corr_transform_to_force(corr_matrix,threshold=threshold)[1]

        # сохраним list_ncorr_features для дальнейшего воспроизведения
        dump(list_ncorr_features, 'models/base/list_ncorr_features/'+'list_ncorr_features_LRST_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # обьявим scaler
        scaler = dict_scalers[scaler]

        # сформируем данные для анализа сбалансированности обучающих данных
        print('Loading train data')
        X_train_pd = fp.ParquetFile('features/base/stat/'+feature_space+'_stat_train').to_pandas()[list_ncorr_features]
        y_train_pd = pd.read_csv('target/target_train.csv')

        # поготовим данные y_train_pd к работе с функцией class_1_percent_samples
        y_train_pd.set_index('id',drop=False,inplace=True)

        # подготовим данные для обучения модели
        # с помощью функции class_1_percent_samples зададим долю 
        # класса 1
        list_c1_percent_id = class_1_percent_samples(y_train_pd,class_1_percent,random_state=random_state)[:1000000]

        # подготовим данные для обучения
        X_train_s_balanced = scaler.fit_transform(X_train_pd.loc[list_c1_percent_id])
        y_train_balanced = y_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

        # сохраним scaler для дальнейшего воспроизведения
        dump(scaler, 'models/base/scalers/'+'scaler_stat_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # освободим память от "тяжелых" и ненужных файлов
        del X_train_pd, y_train_pd
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('start train')
        # обучаем модель LogisticRegression с наилучшеми параметрами
        logistic_regression = linear_model.LogisticRegression(
                C=C,
                solver = solver,
                class_weight=class_weight,
                random_state=random_state,
                max_iter=10000)
        logistic_regression.fit(X_train_s_balanced,y_train_balanced)
        # для того чтобы следить за выполнением цикла введем индикатор
        print('finished train')

        # сохраним модель для дальнейшего воспроизведения
        dump(logistic_regression, 'models/base/'+'LRST_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # удаляем крупные файлы чтобы высвободить память 
        del X_train_s_balanced,y_train_balanced
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Loading test data')
        # подгружаем данные для тестирования
        X_test = fp.ParquetFile('features/base/stat/'+feature_space+'_stat_test').to_pandas()[list_ncorr_features]
        # сформируем данные для проверки модели
        X_test_s = scaler.transform(X_test)
        # для метрик ROC AUC делаем предсказание модели в виде вероятности и записываем в методанные
        features_first_meta_stat['LRST_'+feature_space+'_'+str('0'+str(index))[-2:]] = logistic_regression.predict_proba(X_test_s)[:,1]
        # удаляем крупные файлы чтобы высвободить память 
        del X_test_s, X_test
        gc.collect()
        clear_output()

# сохраним полученные метапризнаки
fp.write('features/firstmeta/features_first_meta_stat',features_first_meta_stat)

##### <span style="color:MediumSlateBlue">Формирование метапризнаков от модели *Gradient Boosting Classifier* обученной сбалансированными *transform data stat* данными</span>

In [25]:
# сформируем метапризнаки для каждого подпространства признаков
for feature_space in dict_spaces.keys():
    # обьявим количество признаков подпространства
    count_features = dict_spaces[feature_space]

    # для работы функции corr_transform_to_force
    # подгрузим DataFrame с матрицей корреляции признаков
    corr_matrix = fp.ParquetFile('features/base/stat/corr_matrix_'+feature_space).to_pandas()

    # загрузим обьект исcледования optuna
    optuna_study_gb_stat = optuna.load_study(study_name='HistGradientBoostingClassifier_'+feature_space+'_stat',
                               storage='sqlite:///optuna_studies.db')
    
    # из полученного обьекта сформируем Data Frame
    optuna_study_gb_stat_pd = optuna_study_gb_stat.trials_dataframe().dropna()

    # переименуем столбы
    optuna_study_gb_stat_pd.rename(columns={
        'values_0': 'roc_train', 
        'values_1': 'roc_valid', 
        'values_2': 'f1_score', 
        'values_3': 'recall_1',
        'values_4': 'precision_1'
    },inplace=True)

    # удалим ненужные столбцы
    optuna_study_gb_stat_pd.drop(['datetime_start','datetime_complete','state','duration'],axis=1, inplace=True)

    # выберем точки с лучшими значениями метрики roc auc на валидационном наборе
    best_roc_gb_stat = optuna_study_gb_stat_pd[optuna_study_gb_stat_pd['roc_valid']>=min_roc_auc].sort_values(by='roc_valid',ascending=False)[:best_models_number] 

    # выберем точки с лучшими значениями метрики precion класса 1 на валидационном наборе
    best_precion_gb_stat = optuna_study_gb_stat_pd[optuna_study_gb_stat_pd['precision_1']>=min_precision].sort_values(by='precision_1',ascending=False)[:best_models_number]   

    # выберем точки в которых максимальна метрика f1-score класса 1 (precision_1=recall_1)
    best_f1_gb_stat = optuna_study_gb_stat_pd[optuna_study_gb_stat_pd['f1_score']>=min_f1].sort_values(by='f1_score',ascending=False)[:best_models_number]

    # сформируем набор признаков для лучших моделей
    best_models_gb_stat = pd.concat([best_roc_gb_stat,best_precion_gb_stat,best_f1_gb_stat]).drop_duplicates().reset_index(drop=True)

    # сформируем список индексов best_models_gb_stat
    list_index = best_models_gb_stat.index

    # сформируем  метапризнаки для обучения метамодели
    # для этого необходиом обучить модель HistGradientBoostingClassifier с выбранными гиперпараметрами
    for index in list_index:

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Current space: ', feature_space)
        print('Number of indexes: ', max(list_index))
        print('Current index: ', index)
        
        # сформируем необходимые параметры
        learning_rate = best_models_gb_stat.loc[index,'params_learning_rate']
        max_iter = best_models_gb_stat.loc[index,'params_max_iter']
        max_leaf_nodes = best_models_gb_stat.loc[index,'params_max_leaf_nodes']
        max_depth = best_models_gb_stat.loc[index,'params_max_depth']
        min_samples_leaf = best_models_gb_stat.loc[index,'params_min_samples_leaf']
        l2_regularization = best_models_gb_stat.loc[index,'params_l2_regularization']
        max_features = best_models_gb_stat.loc[index,'params_max_features']
        class_weight = {0:best_models_gb_stat.loc[index,'params_class_0_weight'],
                        1:best_models_gb_stat.loc[index,'params_class_1_weight']}
        threshold = best_models_gb_stat.loc[index,'params_threshold']
        class_1_percent = best_models_gb_stat.loc[index,'params_class_1_percent']
        random_state = best_models_gb_stat.loc[index,'params_random_state']

        # с помощью функции corr_transform_to_force получим
        # список не коррелируемых признаков
        list_ncorr_features = corr_transform_to_force(corr_matrix,threshold=threshold)[1]

        # сохраним list_ncorr_features для дальнейшего воспроизведения
        dump(list_ncorr_features, 'models/base/list_ncorr_features/'+'list_ncorr_features_GBST_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # сформируем данные для анализа сбалансированности обучающих данных
        print('Loading train data')
        X_train_pd = fp.ParquetFile('features/base/stat/'+feature_space+'_stat_train').to_pandas()[list_ncorr_features]
        y_train_pd = pd.read_csv('target/target_train.csv')

        # поготовим данные y_train_pd к работе с функцией class_1_percent_samples
        y_train_pd.set_index('id',drop=False,inplace=True)

        # подготовим данные для обучения модели
        # с помощью функции class_1_percent_samples зададим долю 
        # класса 1
        list_c1_percent_id = class_1_percent_samples(y_train_pd,class_1_percent,random_state=random_state)[:1000000]

        # подготовим данные для обучения
        X_train_balanced = X_train_pd.loc[list_c1_percent_id]
        y_train_balanced = y_train_pd.loc[list_c1_percent_id]['flag']

        # освободим память от "тяжелых" и ненужных файлов
        del X_train_pd, y_train_pd
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('start train')
        # обучаем модель HistGradientBoostingClassifier с наилучшеми параметрами
        gradient_boosting = HistGradientBoostingClassifier(
                learning_rate = learning_rate,
                max_iter = max_iter,
                max_leaf_nodes = max_leaf_nodes,
                max_depth = max_depth,
                min_samples_leaf = min_samples_leaf,
                l2_regularization = l2_regularization,
                max_features = max_features,
                class_weight = class_weight,
                random_state=42)
        gradient_boosting.fit(X_train_balanced,y_train_balanced)
        # для того чтобы следить за выполнением цикла введем индикатор
        print('finished train')

        # сохраним модель для дальнейшего воспроизведения
        dump(gradient_boosting, 'models/base/'+'GBST_'+feature_space+'_'+str('0'+str(index))[-2:]+'.joblib')

        # удаляем крупные файлы чтобы высвободить память 
        del X_train_balanced,y_train_balanced
        gc.collect()

        # для того чтобы следить за выполнением цикла введем индикатор
        print('Loading test data')
        # подгружаем данные для тестирования
        X_test = fp.ParquetFile('features/base/stat/'+feature_space+'_stat_test').to_pandas()[list_ncorr_features]
        # для метрик ROC AUC делаем предсказание модели в виде вероятности и записываем в методанные
        features_first_meta_stat['GBST_'+feature_space+'_'+str('0'+str(index))[-2:]] = gradient_boosting.predict_proba(X_test)[:,1]
        # удаляем крупные файлы чтобы высвободить память 
        del X_test
        gc.collect()
        clear_output()

# сохраним полученные метапризнаки
fp.write('features/firstmeta/features_first_meta_stat',features_first_meta_stat)

In [26]:
features_first_meta_stat = fp.ParquetFile('features/firstmeta/features_first_meta_stat').to_pandas()
features_first_meta_stat

,id,flag,LRST_date_00,LRST_date_01,LRST_date_02,LRST_date_03,LRST_date_04,LRST_date_05,LRST_date_06,LRST_date_07,...,GBST_service_20,GBST_service_21,GBST_service_22,GBST_service_23,GBST_service_24,GBST_service_25,GBST_service_26,GBST_service_27,GBST_service_28,GBST_service_29
0,1444093,0,0.228598,0.286011,0.306310,0.316349,0.284873,0.305438,0.294386,0.263173,...,0.513262,0.486053,0.516937,0.492899,0.493076,0.509266,0.497604,0.533499,0.521207,0.524906
1,1254992,0,0.207357,0.258086,0.254560,0.263676,0.261145,0.262898,0.287953,0.223825,...,0.442961,0.433920,0.476977,0.394242,0.423567,0.432123,0.409902,0.443103,0.462180,0.477327
2,1792358,0,0.397426,0.444097,0.454559,0.470724,0.459195,0.492430,0.469526,0.400029,...,0.496953,0.504692,0.506555,0.503799,0.503056,0.507792,0.503217,0.531735,0.540401,0.536485
3,712455,0,0.218732,0.291654,0.294577,0.302511,0.284281,0.319613,0.289171,0.257887,...,0.378542,0.378537,0.388864,0.374779,0.370453,0.377065,0.367994,0.411746,0.408434,0.400087
4,1955754,0,0.274674,0.335005,0.351431,0.371115,0.340883,0.375659,0.351630,0.297714,...,0.455626,0.461780,0.465002,0.460135,0.466949,0.461478,0.456959,0.493403,0.489076,0.493711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2137495,423,0,0.214208,0.295340,0.297255,0.300528,0.311420,0.301392,0.316672,0.262315,...,0.294878,0.271255,0.294946,0.294923,0.319590,0.294372,0.304694,0.311352,0.282347,0.313741
2137496,2278100,0,0.259074,0.313675,0.311566,0.323382,0.353013,0.310781,0.322435,0.292823,...,0.218474,0.230880,0.221046,0.240721,0.241160,0.242898,0.227068,0.241012,0.263821,0.261778
2137497,211942,0,0.378021,0.449593,0.461322,0.481972,0.473538,0.467989,0.462929,0.413047,...,0.511684,0.510308,0.515278,0.513717,0.510441,0.509326,0.502068,0.539162,0.541887,0.545609
2137498,2272240,0,0.244668,0.312287,0.319336,0.332818,0.335232,0.331659,0.323031,0.292594,...,0.207959,0.235565,0.231229,0.220327,0.220996,0.230447,0.230896,0.238040,0.240353,0.243872


#### <span style="color:MediumBlue">Формирование тренировочного, валидационного и тестового набора данных</span>

In [27]:
features_first_meta_stat = fp.ParquetFile('features/firstmeta/features_first_meta_stat').to_pandas().iloc[:,:10]

In [28]:
# Выделим из features_first_meta_stat тренировочную часть
MX_train, my_train, MX_test, my_test = my_train_test_split(features_first_meta_stat.set_index('id').drop(['flag'],axis=1),
                                                           features_first_meta_stat.set_index('id')['flag'], train_size=0.35)

In [29]:
# Раздедлим обучающий набор на тренировочную и валидационную части
MX_train, my_train, MX_valid, my_valid = my_train_test_split(MX_train,my_train, train_size=0.8)

Для воспроизводимости кода и корректного сравнения   
различных моделей завиксируем id для тренировочногоб валидационного   
и тестосого наьоров данных

In [30]:
# Спрячем данные в DataDfame для последующего сохраниения в csv
mf_train_id = pd.DataFrame(my_train.index)
mf_valid_id = pd.DataFrame(my_valid.index)
mf_test_id = pd.DataFrame(my_test.index)

In [31]:
# Сохраняем полученные DataFrame
mf_train_id.to_csv('features/firstmeta/mf_train_id.csv')
mf_valid_id.to_csv('features/firstmeta/mf_valid_id.csv')
mf_test_id.to_csv('features/firstmeta/mf_test_id.csv')

In [32]:
# подгрузим DataFrame с id для тренировочного/валидационного/тестового набора данных
mf_train_id = pd.read_csv('features/firstmeta/mf_train_id.csv')['id'].to_list()
mf_valid_id = pd.read_csv('features/firstmeta/mf_valid_id.csv')['id'].to_list()
mf_test_id = pd.read_csv('features/firstmeta/mf_test_id.csv')['id'].to_list()

print(len(mf_train_id))
print(len(mf_valid_id))
print(len(mf_test_id))


598500
149625
1389375


In [33]:
# сохраним тренировочный валидационный и тестовый целевой переменной
# чтобы иметь возможность к ними обратиться
my_train.to_csv('features/firstmeta/mfy_train.csv')
my_valid.to_csv('features/firstmeta/mfy_valid.csv')
my_test.to_csv('features/firstmeta/mfy_test.csv')

#### <span style="color:MediumBlue">Формирование тренировочного, валидационного и тестового набора на features_first_meta_torow данных</span>

In [34]:
features_first_meta_torow = fp.ParquetFile('features/firstmeta/features_first_meta_torow').to_pandas().set_index('id').drop(['flag'],axis=1)
features_first_meta_torow.head(5)

,LRTR_date_00,LRTR_date_01,LRTR_date_02,LRTR_date_03,LRTR_date_04,LRTR_date_05,LRTR_date_06,LRTR_date_07,LRTR_date_08,LRTR_late_00,...,GBTR_service_32,GBTR_service_33,GBTR_service_34,GBTR_service_35,GBTR_service_36,GBTR_service_37,GBTR_service_38,GBTR_service_39,GBTR_service_40,GBTR_service_41
id,,,,,,,,,,,,,,,,,,,,,
1444093,0.186788,0.209557,0.170073,0.193175,0.159265,0.195228,0.246425,0.204595,0.176149,0.028055,...,0.491388,0.455937,0.472278,0.488735,0.444002,0.499014,0.544666,0.510633,0.485606,0.458330
1254992,0.239044,0.259862,0.221379,0.244904,0.189888,0.223594,0.284012,0.298423,0.217998,0.035261,...,0.440126,0.461601,0.385527,0.448575,0.479083,0.387934,0.472670,0.409566,0.366930,0.375410
1792358,0.334716,0.373801,0.290805,0.342053,0.294882,0.341959,0.405733,0.380906,0.320389,0.054827,...,0.503452,0.531439,0.528214,0.529280,0.523508,0.512008,0.512459,0.514849,0.485838,0.503537
712455,0.202599,0.212664,0.188178,0.205899,0.177517,0.207779,0.271352,0.237662,0.191038,0.024818,...,0.374689,0.389504,0.427483,0.408035,0.421883,0.404411,0.384824,0.394359,0.375932,0.388628
1955754,0.299312,0.325026,0.262535,0.299460,0.261249,0.298108,0.370666,0.336419,0.283721,0.039503,...,0.459335,0.480053,0.486514,0.490217,0.471711,0.470499,0.494188,0.460956,0.433987,0.459799


In [35]:
# сохраним тренировочный валидационный и тестовый наборы данных
# чтобы иметь возможность к ними обратиться
fp.write('features/firstmeta/features_first_meta_torow_train',features_first_meta_torow.loc[mf_train_id])
fp.write('features/firstmeta/features_first_meta_torow_valid',features_first_meta_torow.loc[mf_valid_id])
fp.write('features/firstmeta/features_first_meta_torow_test',features_first_meta_torow.loc[mf_test_id])

#### <span style="color:MediumBlue">Формирование тренировочного, валидационного и тестового набора на features_first_meta_stat данных</span>

In [36]:
features_first_meta_stat = fp.ParquetFile('features/firstmeta/features_first_meta_stat').to_pandas().set_index('id').drop(['flag'],axis=1)
features_first_meta_stat.head(5)

,LRST_date_00,LRST_date_01,LRST_date_02,LRST_date_03,LRST_date_04,LRST_date_05,LRST_date_06,LRST_date_07,LRST_late_00,LRST_late_01,...,GBST_service_20,GBST_service_21,GBST_service_22,GBST_service_23,GBST_service_24,GBST_service_25,GBST_service_26,GBST_service_27,GBST_service_28,GBST_service_29
id,,,,,,,,,,,,,,,,,,,,,
1444093,0.228598,0.286011,0.306310,0.316349,0.284873,0.305438,0.294386,0.263173,0.087456,0.091904,...,0.513262,0.486053,0.516937,0.492899,0.493076,0.509266,0.497604,0.533499,0.521207,0.524906
1254992,0.207357,0.258086,0.254560,0.263676,0.261145,0.262898,0.287953,0.223825,0.143654,0.138028,...,0.442961,0.433920,0.476977,0.394242,0.423567,0.432123,0.409902,0.443103,0.462180,0.477327
1792358,0.397426,0.444097,0.454559,0.470724,0.459195,0.492430,0.469526,0.400029,0.105415,0.114457,...,0.496953,0.504692,0.506555,0.503799,0.503056,0.507792,0.503217,0.531735,0.540401,0.536485
712455,0.218732,0.291654,0.294577,0.302511,0.284281,0.319613,0.289171,0.257887,0.071303,0.076709,...,0.378542,0.378537,0.388864,0.374779,0.370453,0.377065,0.367994,0.411746,0.408434,0.400087
1955754,0.274674,0.335005,0.351431,0.371115,0.340883,0.375659,0.351630,0.297714,0.105584,0.114975,...,0.455626,0.461780,0.465002,0.460135,0.466949,0.461478,0.456959,0.493403,0.489076,0.493711


In [37]:
# сохраним тренировочный валидационный и тестовый наборы данных
# чтобы иметь возможность к ними обратиться
fp.write('features/firstmeta/features_first_meta_stat_train',features_first_meta_stat.loc[mf_train_id])
fp.write('features/firstmeta/features_first_meta_stat_valid',features_first_meta_stat.loc[mf_valid_id])
fp.write('features/firstmeta/features_first_meta_stat_test',features_first_meta_stat.loc[mf_test_id])

### <span style="color:RoyalBlue">Построение блендинга из базовых моделей обученных на torow данных</span>

#### <span style="color:MediumBlue">Построение метамодели Logistic Regression</span>

##### <span style="color:MediumSlateBlue">Baseline обучение метамодели LogisticRegression</span>

In [44]:
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv').set_index('id')['flag']
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_torow_valid').to_pandas()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv').set_index('id')['flag']

In [45]:
# обучаем модель логистической регрессии
logistic_regression = linear_model.LogisticRegression(random_state=42, max_iter=10000)
logistic_regression.fit(MX_train,my_train)

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_LR_pred_proba = logistic_regression.predict_proba(MX_train)[:,1]
my_valid_LR_pred_proba = logistic_regression.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_LR_pred = logistic_regression.predict(MX_valid)

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_LR_pred_proba),3))
print('ROC AUC на тестовом наборе', round(metrics.roc_auc_score(my_valid, my_valid_LR_pred_proba),3))

print('Основные метрики на тестовом наборе:')
print(metrics.classification_report(my_valid,my_valid_LR_pred,zero_division=0))

# time: 3m 5s

ROC AUC на обучающем наборе 0.745
ROC AUC на тестовом наборе 0.737
Основные метрики на тестовом наборе:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    144390
           1       0.20      0.00      0.00      5235

    accuracy                           0.96    149625
   macro avg       0.58      0.50      0.49    149625
weighted avg       0.94      0.96      0.95    149625



##### <span style="color:MediumSlateBlue">Подбор гиперпараметров модели</span>

In [46]:
# настроим оптимизацию гипер параметров
def optuna_lg_meta(trial):
  # задаем пространства поиска гиперпараметров
  C = trial.suggest_float('C',0.01,1)
  solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky'])
  class_0_weight = trial.suggest_float('class_0_weight',0.01,1)
  class_1_weight = trial.suggest_float('class_1_weight',0.01,1)
  k_features = trial.suggest_int('k_features', 20, 356,step = 1)
  class_1_percent = trial.suggest_float('class_1_percent',0.1,1)
  random_state = trial.suggest_int('random_state', 1, 1000000)

  # создаем модель
  optuna_log_reg_meta = linear_model.LogisticRegression(
      C=C,
      solver = solver,
      class_weight={0:class_0_weight,1:class_1_weight},
      random_state=random_state,
      max_iter=10000)

  # Загружаем обучающие наборы
  MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()
  my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

  # поготовим данные my_train_pd к работе с функцией class_1_percent_samples
  my_train_pd.set_index('id',drop=False,inplace=True)

  # с помощью функции class_1_percent_samples зададим долю класса 1
  list_c1_percent_id = class_1_percent_samples(my_train_pd,class_1_percent,random_state=random_state)[:1000000]

  # подготовим данные для обучения
  MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
  my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

  # освободим память от "тяжелых" и ненужных файлов
  del MX_train_pd, my_train_pd
  gc.collect()
  
  # с помощью класса SelectKBest получим список лучших признаков
  selector = SelectKBest(f_classif, k=k_features)
  selector.fit(MX_train_balanced, my_train_balanced)
  list_best_features = selector.get_feature_names_out()

  MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()

  # я не воспользовался параметром timeout метода optimize потому, что  
  # optimize отставливает поиск параметров, если время обучения 
  # превышает timeout. Мне нужно чтобы поиск параметров продолжился.
  try:
    # для модуля func_time_out упакуем обучение модели в функцию try_func
    def try_func():
            return optuna_log_reg_meta.fit(MX_train_balanced, my_train_balanced)
    # обучаем модель с ограничением по времени 10 минут (600 секунд)
    optuna_log_reg_meta = func_timeout.func_timeout(600, try_func)

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()

    # загружаем тестовые наборы
    MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()[list_best_features].to_numpy()
    MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_torow_valid').to_pandas()[list_best_features].to_numpy()
    my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
    my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()
    
    # делаем предсказание на обучающем и валидационном наборе и считаем метрики
    roc_train = metrics.roc_auc_score(my_train, optuna_log_reg_meta.predict_proba(MX_train)[:,1])
    roc_valid = metrics.roc_auc_score(my_valid, optuna_log_reg_meta.predict_proba(MX_valid)[:,1])
    f1_score = metrics.f1_score(my_valid, optuna_log_reg_meta.predict(MX_valid))
    recall_1 = metrics.recall_score(my_valid, optuna_log_reg_meta.predict(MX_valid),average=None,zero_division=0)[1]
    precision_1 = metrics.precision_score(my_valid, optuna_log_reg_meta.predict(MX_valid),average=None,zero_division=0)[1]

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train, MX_valid, my_train, my_valid
    gc.collect()

  except func_timeout.FunctionTimedOut:
    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()
    
    # фиксируем пустые значения метрик
    roc_train = 0
    roc_valid = 0
    f1_score = 0
    recall_1 = 0
    precision_1 = 0
    pass

  return roc_train, roc_valid, f1_score, recall_1, precision_1

In [ ]:
# optuna.delete_study(study_name="LogisticRegression_meta_first_torow", storage='sqlite:///optuna_studies.db')
# cоздаем объект исследования
# чтобы модель не переобучалась минимизируем roc_train 
# и максимизируем roc_valid
optuna_study_lg_meta = optuna.create_study(study_name="LogisticRegression_meta_first_torow", 
                               directions=['maximize','maximize','maximize','maximize','maximize'], 
                               sampler=optuna.samplers.TPESampler(),
                               pruner='Hyperband',
                               storage='sqlite:///optuna_studies.db',
                               load_if_exists=True)

[I 2025-04-20 17:36:24,723] A new study created in RDB with name: LogisticRegression_meta_first_torow


In [48]:
# ищем лучшую комбинацию гиперпараметров n_trials раз
optuna_study_lg_meta.optimize(optuna_lg_meta, n_trials=300)

[I 2025-04-20 17:36:27,553] Trial 0 finished with values: [0.6940356348372834, 0.6868307267414151, 0.00038182512409316535, 0.00019102196752626553, 0.3333333333333333] and parameters: {'C': 0.11031179816286364, 'solver': 'newton-cholesky', 'class_0_weight': 0.9880259990732702, 'class_1_weight': 0.21536603755772335, 'k_features': 22, 'class_1_percent': 0.34068582071795245, 'random_state': 61752}.
[I 2025-04-20 17:36:31,128] Trial 1 finished with values: [0.7460690025295422, 0.7364510026139675, 0.06777926066055949, 0.9992359121298949, 0.03507936614381803] and parameters: {'C': 0.578161766436431, 'solver': 'newton-cholesky', 'class_0_weight': 0.0970159878062788, 'class_1_weight': 0.2038793595030669, 'k_features': 319, 'class_1_percent': 0.8170444508108471, 'random_state': 451614}.
[I 2025-04-20 17:36:35,731] Trial 2 finished with values: [0.7498096565160053, 0.7378256027249768, 0.1536723163841808, 0.16886341929321871, 0.14098883572567783] and parameters: {'C': 0.6171479788308843, 'solver':

##### <span style="color:MediumSlateBlue">Анализ гиперпараметров</span>

In [49]:
# из полученного результат соврмируем Data Frame
optuna_study_lg_meta_pd = optuna_study_lg_meta.trials_dataframe().dropna()
# переименуем столбы
optuna_study_lg_meta_pd.rename(columns={
    'values_0': 'roc_train',
    'values_1': 'roc_valid',
    'values_2': 'f1_score',
    'values_3': 'recall_1',
    'values_4': 'precision_1'
},inplace=True)
# Выделим время потраченное на обучение модели
optuna_study_lg_meta_pd.tail()

,number,roc_train,roc_valid,f1_score,recall_1,precision_1,datetime_start,datetime_complete,duration,params_C,params_class_0_weight,params_class_1_percent,params_class_1_weight,params_k_features,params_random_state,params_solver,state
295,295,0.750848,0.740206,0.158317,0.404776,0.098403,2025-04-20 18:01:26.242190,2025-04-20 18:01:33.366617,0 days 00:00:07.124427,0.802815,0.256365,0.395373,0.212852,341,611801,liblinear,COMPLETE
296,296,0.742962,0.730683,0.088609,0.923782,0.046536,2025-04-20 18:01:33.371641,2025-04-20 18:01:38.538392,0 days 00:00:05.166751,0.815527,0.230913,0.429195,0.777775,190,238543,liblinear,COMPLETE
297,297,0.751009,0.739289,0.085700,0.939828,0.044897,2025-04-20 18:01:38.544440,2025-04-20 18:01:46.703088,0 days 00:00:08.158648,0.858042,0.149071,0.366052,0.754079,328,584230,liblinear,COMPLETE
298,298,0.750584,0.738551,0.121917,0.714231,0.066646,2025-04-20 18:01:46.708757,2025-04-20 18:01:53.852193,0 days 00:00:07.143436,0.792399,0.172989,0.397806,0.304821,322,242512,liblinear,COMPLETE
299,299,0.750508,0.738972,0.151633,0.480229,0.090030,2025-04-20 18:01:53.857429,2025-04-20 18:02:00.498482,0 days 00:00:06.641053,0.815738,0.245214,0.415227,0.220710,342,584885,liblinear,COMPLETE


In [50]:
# покаждем статистику обучения
print('Максимальное значение метрики ROC AUC на валидационном наборе:',round(optuna_study_lg_meta_pd['roc_valid'].max(),3))
print('Среднее значение метрики ROC AUC на валидационном наборе:',round(optuna_study_lg_meta_pd['roc_valid'].mean(),3))
print('Максимальное значение метрики f1_score на валидационном наборе:',round(optuna_study_lg_meta_pd['f1_score'].max(),3))
print('Среднее значение метрики f1_score на валидационном наборе:',round(optuna_study_lg_meta_pd['f1_score'].mean(),3))
print('Максимальное значение метрики recall_1 на валидационном наборе:',round(optuna_study_lg_meta_pd['recall_1'].max(),3))
print('Среднее значение метрики recall_1 на валидационном наборе:',round(optuna_study_lg_meta_pd['recall_1'].mean(),3))
print('Максимальное значение метрики precision_1 на валидационном наборе:',round(optuna_study_lg_meta_pd['precision_1'].max(),3))
print('Среднее значение метрики precision_1 на валидационном наборе:',round(optuna_study_lg_meta_pd['precision_1'].mean(),3))

Максимальное значение метрики ROC AUC на валидационном наборе: 0.741
Среднее значение метрики ROC AUC на валидационном наборе: 0.737
Максимальное значение метрики f1_score на валидационном наборе: 0.166
Среднее значение метрики f1_score на валидационном наборе: 0.124
Максимальное значение метрики recall_1 на валидационном наборе: 1.0
Среднее значение метрики recall_1 на валидационном наборе: 0.466
Максимальное значение метрики precision_1 на валидационном наборе: 0.368
Среднее значение метрики precision_1 на валидационном наборе: 0.101


In [52]:
# построим график важности гиперпарметров
optuna.visualization.plot_param_importances(optuna_study_lg_meta, target_name='Score')

In [53]:
# Построим зависимость метрики precision_1 от гипер параметров

fig = px.scatter(
    data_frame=optuna_study_lg_meta_pd,
    x='precision_1', #ось абсцисс
    y=['f1_score','roc_train', 'roc_valid', 'recall_1'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость precision от гиперпараметров модели', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='Величина метрики precision_1',
    yaxis_title='Величина параметра')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

In [102]:
# Визуализируем метрики полученные при optuna оптимизации
fig = px.scatter(
    data_frame=optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['roc_valid']>0.9999*optuna_study_lg_meta_pd['roc_valid'].max()],
    x='number', #ось абсцисс
    y=['roc_train','roc_valid','recall_1','precision_1'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость качества модели от trials optuna', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='trials optuna',
    yaxis_title='Величина метрики')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

<span style="color:Blue">

Вывод:  

Их всех выбираем точку $number =213$.   
В этой точке самые оптимальные значения $recall$ и $precision$.  
Посмотрим каким значениям гиперпараметров соотвествует выбранная точка.

In [103]:
# определим номер лучшге варианта
best_optuna_number = 213

# сформируем словарь лучших гипирпарметров RandomForestClassifier
best_param_lr = {
    'solver' : optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['params_solver'].iloc[0],
    'C' : round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['params_C'].iloc[0],3),
    'class_weight' : {0:round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['params_class_0_weight'].iloc[0],3),
                      1:round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['params_class_1_weight'].iloc[0],3)},
    }

# создадим перменные
best_k_features = optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['params_k_features'].iloc[0]
best_random_state = optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['params_random_state'].iloc[0]
best_class_1_percent = optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['params_class_1_percent'].iloc[0]

# Выведем принятые наилучшие праметры
print('best solver:',best_param_lr['solver'])
print('best C:',best_param_lr['C'])
print('best class 0 weight:',best_param_lr['class_weight'][0])
print('best class 1 weight:',best_param_lr['class_weight'][1])

print('best class 1 percent:',round(best_class_1_percent,3))
print('best k features:',best_k_features)
print('best random state:',best_random_state)
print('time for best train:',round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['duration'].iloc[0].seconds/60),'minutes')

print()
print('ROC AUC на обучающем наборе:', round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['roc_train'].iloc[0],3))
print('ROC AUC на валидационном наборе:', round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['roc_valid'].iloc[0],3))
print('precision класса 1:', round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['precision_1'].iloc[0],3))
print('recall класса 1:', round(optuna_study_lg_meta_pd[optuna_study_lg_meta_pd['number']==best_optuna_number]['recall_1'].iloc[0],3))


best solver: liblinear
best C: 0.501
best class 0 weight: 0.775
best class 1 weight: 0.781
best class 1 percent: 0.358
best k features: 350
best random state: 332173
time for best train: 0 minutes

ROC AUC на обучающем наборе: 0.752
ROC AUC на валидационном наборе: 0.741
precision класса 1: 0.098
recall класса 1: 0.428


##### <span style="color:MediumSlateBlue">Обучение модели с лучшими параметрами</span>

In [104]:
# Загружаем обучающие наборы
MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()
my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

# поготовим данные my_train_pd к работе с функцией class_1_percent_samples
my_train_pd.set_index('id',drop=False,inplace=True)

# с помощью функции class_1_percent_samples зададим долю класса 1
list_c1_percent_id = class_1_percent_samples(my_train_pd,best_class_1_percent,random_state=best_random_state)[:1000000]

# подготовим данные для обучения
MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

# освободим память от "тяжелых" и ненужных файлов
del MX_train_pd, my_train_pd
gc.collect()

# с помощью класса SelectKBest получим список лучших признаков
selector = SelectKBest(f_classif, k=best_k_features)
selector.fit(MX_train_balanced, my_train_balanced)
list_best_features = selector.get_feature_names_out()

MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()


# обучаем модель LogisticRegression с наилучшеми параметрами
logistic_regression = linear_model.LogisticRegression(
        **best_param_lr,
        random_state=best_random_state,
        max_iter=10000)
logistic_regression.fit(MX_train_balanced, my_train_balanced)

# удаляем крупные файлы чтобы высвободить память 
del MX_train_balanced, my_train_balanced
gc.collect()

# загружаем тестовые наборы
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()[list_best_features].to_numpy()
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_torow_valid').to_pandas()[list_best_features].to_numpy()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_lr_pred_proba = logistic_regression.predict_proba(MX_train)[:,1]
my_valid_lr_pred_proba = logistic_regression.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_lr_pred = logistic_regression.predict(MX_valid)

# удаляем крупные файлы чтобы высвободить память 
del MX_train, MX_valid
gc.collect()

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_lr_pred_proba),3))
print('ROC AUC на валидационном наборе', round(metrics.roc_auc_score(my_valid, my_valid_lr_pred_proba),3))

print('Основные метрики на валидационом наборе:')
print(metrics.classification_report(my_valid,my_valid_lr_pred,zero_division=0))

# time: 2m 30s

ROC AUC на обучающем наборе 0.752
ROC AUC на валидационном наборе 0.741
Основные метрики на валидационом наборе:
              precision    recall  f1-score   support

           0       0.98      0.86      0.91    144390
           1       0.10      0.43      0.16      5235

    accuracy                           0.84    149625
   macro avg       0.54      0.64      0.54    149625
weighted avg       0.95      0.84      0.89    149625



#### <span style="color:MediumBlue">Построение модели Gradient Boosting Classifier</span>

##### <span style="color:MediumSlateBlue">Baseline обучение модели Hist Gradient Boosting Classifier</span>

In [57]:
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv').set_index('id')['flag']
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_torow_valid').to_pandas()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv').set_index('id')['flag']

In [58]:
# обучаем модель Gradient Boosting Classifier
gradient_boosting = HistGradientBoostingClassifier(random_state=42)
gradient_boosting.fit(MX_train,my_train)

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_rf__pred_proba = gradient_boosting.predict_proba(MX_train)[:,1]
my_valid_rf_pred_proba = gradient_boosting.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_rf_pred = gradient_boosting.predict(MX_valid)

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_rf__pred_proba),3))
print('ROC AUC на валидационном наборе', round(metrics.roc_auc_score(my_valid, my_valid_rf_pred_proba),3))

print('Основные метрики на валидационом наборе:')
print(metrics.classification_report(my_valid,my_valid_rf_pred))

# time: 30s

ROC AUC на обучающем наборе 0.792
ROC AUC на валидационном наборе 0.743
Основные метрики на валидационом наборе:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    144390
           1       0.00      0.00      0.00      5235

    accuracy                           0.96    149625
   macro avg       0.48      0.50      0.49    149625
weighted avg       0.93      0.96      0.95    149625



<span style="color:Blue">

Выводы:

В сравнение с моделью $HistGradientBoostingClassifier$ построенной   
на несбалансированных данных $transform$ $data$ $stat$:
1. Качество модели по метрике $ROC AUC$ не изменилась.
2. Качество модели по метрике $recall_1$ значительно  
улучшилось. Правда, $recall_0$ уменьшилось.
3. Качество модели по метрике $precision_1$ уменьшилось.   
4. На порядок уменьшилось время обучения модели. Это логично,  
велична сбалансированной выборки всего $107000$ $samples$. 

##### <span style="color:MediumSlateBlue">Подбор гиперпараметров модели</span>

In [59]:
# настроим оптимизацию гипер параметров
def optuna_gb_meta(trial):
  # задаем пространства поиска гиперпараметров
  learning_rate = trial.suggest_float('learning_rate',0.01,0.1)
  max_iter = trial.suggest_int('max_iter', 150, 250,step = 1)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 60,step = 1)
  max_depth = trial.suggest_int('max_depth', 1, 10,step = 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 60,step = 1)
  max_features = trial.suggest_float('max_features',0.1,1)
  l2_regularization = trial.suggest_float('l2_regularization',0.01,1)
  class_0_weight = trial.suggest_float('class_0_weight',0.01,1)
  class_1_weight = trial.suggest_float('class_1_weight',0.01,1)
  k_features = trial.suggest_int('k_features', 20, 356,step = 1)
  class_1_percent = trial.suggest_float('class_1_percent',0.01,1)
  random_state = trial.suggest_int('random_state', 1, 1000000)

  # создаем модель
  optuna_gradient_boosting_meta = HistGradientBoostingClassifier(
      learning_rate= learning_rate,
      max_iter = max_iter,
      max_leaf_nodes =max_leaf_nodes,
      max_depth = max_depth,
      min_samples_leaf = min_samples_leaf,
      max_features=max_features,
      l2_regularization = l2_regularization,
      class_weight={0:class_0_weight,1:class_1_weight},
      random_state=random_state)

  # Загружаем обучающие наборы
  MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()
  my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

  # поготовим данные my_train_pd к работе с функцией class_1_percent_samples
  my_train_pd.set_index('id',drop=False,inplace=True)

  # с помощью функции class_1_percent_samples зададим долю класса 1
  list_c1_percent_id = class_1_percent_samples(my_train_pd,class_1_percent,random_state=random_state)[:1000000]

  # подготовим данные для обучения
  MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
  my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

  # освободим память от "тяжелых" и ненужных файлов
  del MX_train_pd, my_train_pd
  gc.collect()
  
  # с помощью класса SelectKBest получим список лучших признаков
  selector = SelectKBest(f_classif, k=k_features)
  selector.fit(MX_train_balanced, my_train_balanced)
  list_best_features = selector.get_feature_names_out()

  MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()

  # я не воспользовался параметром timeout метода optimize потому, что  
  # optimize отставливает поиск параметров, если время обучения 
  # превышает timeout. Мне нужно чтобы поиск параметров продолжился.
  try:
    # для модуля func_time_out упакуем обучение модели в функцию try_func
    def try_func():
            return optuna_gradient_boosting_meta.fit(MX_train_balanced,my_train_balanced)
    # обучаем модель с ограничением по времени 10 минут (600 секунд)
    optuna_gradient_boosting_meta = func_timeout.func_timeout(600, try_func)

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()

    # загружаем тестовые наборы
    MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()[list_best_features].to_numpy()
    MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_torow_valid').to_pandas()[list_best_features].to_numpy()
    my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
    my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()

    # делаем предсказание на обучающем и валидационном наборе
    #Считаем метрики для класса 1 добавляем их в список
    roc_train = metrics.roc_auc_score(my_train, optuna_gradient_boosting_meta.predict_proba(MX_train)[:,1])
    roc_valid = metrics.roc_auc_score(my_valid, optuna_gradient_boosting_meta.predict_proba(MX_valid)[:,1])
    f1_score = metrics.f1_score(my_valid, optuna_gradient_boosting_meta.predict(MX_valid))
    recall_1 = metrics.recall_score(my_valid, optuna_gradient_boosting_meta.predict(MX_valid),average=None,zero_division=0)[1]
    precision_1 = metrics.precision_score(my_valid, optuna_gradient_boosting_meta.predict(MX_valid),average=None,zero_division=0)[1]

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train, MX_valid, my_train, my_valid
    gc.collect()

  except func_timeout.FunctionTimedOut:
    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()
    
    # фиксируем пустые значения метрик
    roc_train = 0
    roc_valid = 0
    f1_score = 0
    recall_1 = 0
    precision_1 = 0
    pass

  return roc_train, roc_valid, f1_score, recall_1, precision_1

In [ ]:
# ну случай удаления обучения
# optuna.delete_study(study_name="HistGradientBoostingClassifier_meta_first_torow", storage='sqlite:///optuna_studies.db')

# cоздаем объект исследования
# чтобы модель не переобучалась минимизируем roc_train 
# и максимизируем roc_valid
optuna_study_gb_meta = optuna.create_study(study_name="HistGradientBoostingClassifier_meta_first_torow", 
                               directions=['maximize','maximize','maximize','maximize','maximize'], 
                               sampler=optuna.samplers.TPESampler(),
                               pruner='Hyperband',
                               storage='sqlite:///optuna_studies.db',
                               load_if_exists=True)

[I 2025-04-20 18:02:29,826] A new study created in RDB with name: HistGradientBoostingClassifier_meta_first_torow


In [61]:
# ищем лучшую комбинацию гиперпараметров n_trials раз
optuna_study_gb_meta.optimize(optuna_gb_meta, n_trials=300)

[I 2025-04-20 18:02:38,184] Trial 0 finished with values: [0.762448582997421, 0.7356776745407168, 0.15951228709697693, 0.24240687679083095, 0.11886474334956913] and parameters: {'learning_rate': 0.024006948967409406, 'max_iter': 155, 'max_leaf_nodes': 47, 'max_depth': 5, 'min_samples_leaf': 19, 'max_features': 0.556756545780574, 'l2_regularization': 0.3585916218101101, 'class_0_weight': 0.13073082039027087, 'class_1_weight': 0.187417901374876, 'k_features': 220, 'class_1_percent': 0.21036236538128686, 'random_state': 432244}.
[I 2025-04-20 18:02:41,541] Trial 1 finished with values: [0.7488905365136755, 0.7310752814279855, 0.07832522601251286, 0.9673352435530086, 0.040815010638983816] and parameters: {'learning_rate': 0.08828844931388287, 'max_iter': 246, 'max_leaf_nodes': 43, 'max_depth': 3, 'min_samples_leaf': 38, 'max_features': 0.663617124104108, 'l2_regularization': 0.3525184194811143, 'class_0_weight': 0.4110844627359958, 'class_1_weight': 0.5713791849718606, 'k_features': 160, '

##### <span style="color:MediumSlateBlue">Анализ гиперпараметров</span>

In [62]:
# из полученного результат соврмируем Data Frame
optuna_study_gb_meta_pd = optuna_study_gb_meta.trials_dataframe()
# переименуем столбы
optuna_study_gb_meta_pd.rename(columns={
    'values_0': 'roc_train',
    'values_1': 'roc_valid',
    'values_2': 'f1_score',
    'values_3': 'recall_1',
    'values_4': 'precision_1'
},inplace=True)
optuna_study_gb_meta_pd.tail(5)

,number,roc_train,roc_valid,f1_score,recall_1,precision_1,datetime_start,datetime_complete,duration,params_class_0_weight,...,params_k_features,params_l2_regularization,params_learning_rate,params_max_depth,params_max_features,params_max_iter,params_max_leaf_nodes,params_min_samples_leaf,params_random_state,state
295,295,0.795564,0.744243,0.149774,0.164852,0.137224,2025-04-20 18:51:03.333111,2025-04-20 18:51:16.581460,0 days 00:00:13.248349,0.651024,...,339,0.277669,0.081731,8,0.328420,194,59,38,76466,COMPLETE
296,296,0.761505,0.742570,0.163037,0.225215,0.127763,2025-04-20 18:51:16.587624,2025-04-20 18:51:30.921314,0 days 00:00:14.333690,0.632323,...,349,0.255832,0.019345,4,0.317473,203,59,34,76061,COMPLETE
297,297,0.798146,0.746495,0.168336,0.282521,0.119883,2025-04-20 18:51:30.927423,2025-04-20 18:51:50.543477,0 days 00:00:19.616054,0.669912,...,341,0.346893,0.032010,8,0.337752,204,60,35,59535,COMPLETE
298,298,0.801353,0.746268,0.159223,0.186246,0.139047,2025-04-20 18:51:50.549528,2025-04-20 18:52:13.807980,0 days 00:00:23.258452,0.662894,...,329,0.340077,0.025337,8,0.619916,209,60,36,47402,COMPLETE
299,299,0.783417,0.745474,0.166339,0.258453,0.122632,2025-04-20 18:52:13.814542,2025-04-20 18:52:37.945596,0 days 00:00:24.131054,0.627800,...,339,0.343936,0.016333,8,0.165393,203,59,32,68738,COMPLETE


In [63]:
# покаждем статистику обучения
print('Максимальное значение метрики ROC AUC на валидационном наборе:',round(optuna_study_gb_meta_pd['roc_valid'].max(),3))
print('Среднее значение метрики ROC AUC на валидационном наборе:',round(optuna_study_gb_meta_pd['roc_valid'].mean(),3))
print('Максимальное значение метрики f1_score на валидационном наборе:',round(optuna_study_gb_meta_pd['f1_score'].max(),3))
print('Среднее значение метрики f1_score на валидационном наборе:',round(optuna_study_gb_meta_pd['f1_score'].mean(),3))
print('Максимальное значение метрики recall_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['recall_1'].max(),3))
print('Среднее значение метрики recall_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['recall_1'].mean(),3))
print('Максимальное значение метрики precision_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['precision_1'].max(),3))
print('Среднее значение метрики precision_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['precision_1'].mean(),3))

Максимальное значение метрики ROC AUC на валидационном наборе: 0.747
Среднее значение метрики ROC AUC на валидационном наборе: 0.741
Максимальное значение метрики f1_score на валидационном наборе: 0.169
Среднее значение метрики f1_score на валидационном наборе: 0.124
Максимальное значение метрики recall_1 на валидационном наборе: 1.0
Среднее значение метрики recall_1 на валидационном наборе: 0.436
Максимальное значение метрики precision_1 на валидационном наборе: 0.75
Среднее значение метрики precision_1 на валидационном наборе: 0.111


In [65]:
# построим график важности гиперпарметров
optuna.visualization.plot_param_importances(optuna_study_gb_meta, target_name='Score')

In [66]:
# Построим зависимость метрики precision_1 от гиперпараметров
fig = px.scatter(
    data_frame=optuna_study_gb_meta_pd,
    x='precision_1', #ось абсцисс
    y=['roc_train', 'roc_valid', 'recall_1','f1_score'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость precision класса 1 от гиперпараметров модели', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='Величина метрики precision класса 1',
    yaxis_title='Величина параметра')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

In [107]:
# Построим зависимость метрик качества модели от number

fig = px.scatter(
    data_frame=optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['roc_valid']>0.9999*(optuna_study_gb_meta_pd['roc_valid'].max())],
    x='number', #ось абсцисс
    y=['roc_train','roc_valid','recall_1','precision_1'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость precision от гиперпараметров модели', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='Величина метрики precision_1',
    yaxis_title='Величина параметра')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

<span style="color:Blue">

Вывод:  

Их всех выбираем точку $number =272$.   
В этой точке оптимальные значения метрик $recall_1$ и $precision_1$, а  
также относительно высокая метрика $ROC AUC$.   
Посмотрим каким значениям гиперпараметров соотвествует выбранная точка.

In [108]:
# определим номер лучшге варианта
best_optuna_number = 272

# сформируем словарь лучших гипирпарметров HistGradientBoostingClassifier
best_param_hgbc = {
    'learning_rate' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_learning_rate'].iloc[0],
    'max_iter' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_iter'].iloc[0],
    'max_leaf_nodes' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_leaf_nodes'].iloc[0],
    'max_depth' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_depth'].iloc[0],
    'min_samples_leaf' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_min_samples_leaf'].iloc[0],
    'max_features' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_features'].iloc[0],
    'l2_regularization' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_l2_regularization'].iloc[0],
    'class_weight' : {0: optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_class_0_weight'].iloc[0],
                      1: optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_class_1_weight'].iloc[0],}
    }

# определим перменные для лучших значений параметров
best_k_features = optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_k_features'].iloc[0]
best_random_state = optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_random_state'].iloc[0]
best_class_1_percent = optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_class_1_percent'].iloc[0]


# Выведем принятые наилучшие параметры
print('best learning rate:',round(best_param_hgbc['learning_rate'],3))
print('best max iter:',best_param_hgbc['max_iter'])
print('best max leaf nodes:',best_param_hgbc['max_leaf_nodes'])
print('best max depth:',best_param_hgbc['max_depth'])
print('best min samples leaf:',best_param_hgbc['min_samples_leaf'])
print('best max features:',round(best_param_hgbc['max_features'],3))
print('best l2 regularization:',round(best_param_hgbc['l2_regularization'],3))
print('best class 0 weight:',round(best_param_hgbc['class_weight'][0],3))
print('best class 1 weight:',round(best_param_hgbc['class_weight'][1],3))

print('best class 1 percent:',round(best_class_1_percent,3))
print('best k features:',best_k_features)
print('best random state:',best_random_state)
print('time for best train:',round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['duration'].iloc[0].seconds/60),'minutes')

print()
print('ROC AUC на обучающем наборе:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['roc_train'].iloc[0],3))
print('ROC AUC на валидационном наборе:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['roc_valid'].iloc[0],3))
print('precision класса 1:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['precision_1'].iloc[0],3))
print('recall класса 1:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['recall_1'].iloc[0],3))

best learning rate: 0.025
best max iter: 232
best max leaf nodes: 58
best max depth: 9
best min samples leaf: 7
best max features: 0.346
best l2 regularization: 0.688
best class 0 weight: 0.233
best class 1 weight: 0.616
best class 1 percent: 0.15
best k features: 334
best random state: 975267
time for best train: 0 minutes

ROC AUC на обучающем наборе: 0.788
ROC AUC на валидационном наборе: 0.736
precision класса 1: 0.103
recall класса 1: 0.333


##### <span style="color:MediumSlateBlue">Обучение модели с лучшими параметрами</span>

In [109]:
# Загружаем обучающие наборы
MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()
my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

# поготовим данные my_train_pd к работе с функцией class_1_percent_samples
my_train_pd.set_index('id',drop=False,inplace=True)

# с помощью функции class_1_percent_samples зададим долю класса 1
list_c1_percent_id = class_1_percent_samples(my_train_pd,best_class_1_percent,random_state=best_random_state)[:1000000]

# подготовим данные для обучения
MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

# освободим память от "тяжелых" и ненужных файлов
del MX_train_pd, my_train_pd
gc.collect()

# с помощью класса SelectKBest получим список лучших признаков
selector = SelectKBest(f_classif, k=best_k_features)
selector.fit(MX_train_balanced, my_train_balanced)
list_best_features = selector.get_feature_names_out()

MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()

# обучаем модель HistGradientBoostingClassifier с наилучшеми параметрами
gradient_boosting = HistGradientBoostingClassifier(
        **best_param_hgbc,
        random_state=best_random_state)
gradient_boosting.fit(MX_train_balanced,my_train_balanced)

# удаляем крупные файлы чтобы высвободить память 
del MX_train_balanced, my_train_balanced
gc.collect()

# загружаем тестовые наборы
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_torow_train').to_pandas()[list_best_features].to_numpy()
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_torow_valid').to_pandas()[list_best_features].to_numpy()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_gb_pred_proba = gradient_boosting.predict_proba(MX_train)[:,1]
my_valid_gb_pred_proba = gradient_boosting.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_gb_pred = gradient_boosting.predict(MX_valid)

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_gb_pred_proba),3))
print('ROC AUC на валидационном наборе', round(metrics.roc_auc_score(my_valid, my_valid_gb_pred_proba),3))

print('Основные метрики на валидационом наборе:')
print(metrics.classification_report(my_valid,my_valid_gb_pred,zero_division=0))

# time: 1m 40s

ROC AUC на обучающем наборе 0.801
ROC AUC на валидационном наборе 0.747
Основные метрики на валидационом наборе:
              precision    recall  f1-score   support

           0       0.97      0.89      0.93    144390
           1       0.11      0.37      0.16      5235

    accuracy                           0.87    149625
   macro avg       0.54      0.63      0.55    149625
weighted avg       0.94      0.87      0.90    149625



### <span style="color:RoyalBlue">Построение блендинга из базовых моделей обученных на transform data stat данных</span>

#### <span style="color:MediumBlue">Построение метамодели Logistic Regression</span>

##### <span style="color:MediumSlateBlue">Baseline обучение метамодели LogisticRegression</span>

In [70]:
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv').set_index('id')['flag']
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_stat_valid').to_pandas()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv').set_index('id')['flag']

In [71]:
# обучаем модель логистической регрессии
logistic_regression = linear_model.LogisticRegression(random_state=42, max_iter=10000)
logistic_regression.fit(MX_train,my_train)

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_LR_pred_proba = logistic_regression.predict_proba(MX_train)[:,1]
my_valid_LR_pred_proba = logistic_regression.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_LR_pred = logistic_regression.predict(MX_valid)

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_LR_pred_proba),3))
print('ROC AUC на тестовом наборе', round(metrics.roc_auc_score(my_valid, my_valid_LR_pred_proba),3))

print('Основные метрики на тестовом наборе:')
print(metrics.classification_report(my_valid,my_valid_LR_pred,zero_division=0))

# time: 3m 5s

ROC AUC на обучающем наборе 0.736
ROC AUC на тестовом наборе 0.727
Основные метрики на тестовом наборе:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    144390
           1       0.13      0.00      0.00      5235

    accuracy                           0.96    149625
   macro avg       0.55      0.50      0.49    149625
weighted avg       0.94      0.96      0.95    149625



##### <span style="color:MediumSlateBlue">Подбор гиперпараметров модели</span>

In [72]:
# настроим оптимизацию гипер параметров
def optuna_lg_meta(trial):
  # задаем пространства поиска гиперпараметров
  C = trial.suggest_float('C',0.01,1)
  class_0_weight = trial.suggest_float('class_0_weight',0.01,1)
  class_1_weight = trial.suggest_float('class_1_weight',0.01,1)
  k_features = trial.suggest_int('k_features', 20, 342,step = 1)
  class_1_percent = trial.suggest_float('class_1_percent',0.1,1)
  random_state = trial.suggest_int('random_state', 1, 1000000)


  # создаем модель
  optuna_log_reg_meta = linear_model.LogisticRegression(
      C=C,
      class_weight={0:class_0_weight,1:class_1_weight},
      random_state=random_state,
      max_iter=10000)

  # Загружаем обучающие наборы
  MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()
  my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

  # поготовим данные my_train_pd к работе с функцией class_1_percent_samples
  my_train_pd.set_index('id',drop=False,inplace=True)

  # с помощью функции class_1_percent_samples зададим долю класса 1
  list_c1_percent_id = class_1_percent_samples(my_train_pd,class_1_percent,random_state=random_state)[:1000000]

  # подготовим данные для обучения
  MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
  my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

  # освободим память от "тяжелых" и ненужных файлов
  del MX_train_pd, my_train_pd
  gc.collect()
  
  # с помощью класса SelectKBest получим список лучших признаков
  selector = SelectKBest(f_classif, k=k_features)
  selector.fit(MX_train_balanced, my_train_balanced)
  list_best_features = selector.get_feature_names_out()

  MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()

  # я не воспользовался параметром timeout метода optimize потому, что  
  # optimize отставливает поиск параметров, если время обучения 
  # превышает timeout. Мне нужно чтобы поиск параметров продолжился.
  try:
    # для модуля func_time_out упакуем обучение модели в функцию try_func
    def try_func():
            return optuna_log_reg_meta.fit(MX_train_balanced, my_train_balanced)
    # обучаем модель с ограничением по времени 10 минут (600 секунд)
    optuna_log_reg_meta = func_timeout.func_timeout(600, try_func)

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()

    # загружаем тестовые наборы
    MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()[list_best_features].to_numpy()
    MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_stat_valid').to_pandas()[list_best_features].to_numpy()
    my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
    my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()
    
    # делаем предсказание на обучающем и валидационном наборе и считаем метрики
    roc_train = metrics.roc_auc_score(my_train, optuna_log_reg_meta.predict_proba(MX_train)[:,1])
    roc_valid = metrics.roc_auc_score(my_valid, optuna_log_reg_meta.predict_proba(MX_valid)[:,1])
    f1_score = metrics.f1_score(my_valid, optuna_log_reg_meta.predict(MX_valid))
    recall_1 = metrics.recall_score(my_valid, optuna_log_reg_meta.predict(MX_valid),average=None,zero_division=0)[1]
    precision_1 = metrics.precision_score(my_valid, optuna_log_reg_meta.predict(MX_valid),average=None,zero_division=0)[1]

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train, MX_valid, my_train, my_valid
    gc.collect()

  except func_timeout.FunctionTimedOut:
    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()
    
    # фиксируем пустые значения метрик
    roc_train = 0
    roc_valid = 0
    f1_score = 0
    recall_1 = 0
    precision_1 = 0
    pass

  return roc_train, roc_valid, f1_score, recall_1, precision_1

In [ ]:
# на случай удаления
# optuna.delete_study(study_name="LogisticRegression_meta_first_stat", storage='sqlite:///optuna_studies.db')

# cоздаем объект исследования
# чтобы модель не переобучалась минимизируем roc_train 
# и максимизируем roc_valid
optuna_study_lg_meta = optuna.create_study(study_name="LogisticRegression_meta_first_stat", 
                               directions=['maximize','maximize','maximize','maximize','maximize'], 
                               sampler=optuna.samplers.TPESampler(),
                               pruner='Hyperband',
                               storage='sqlite:///optuna_studies.db',
                               load_if_exists=True)

[I 2025-04-20 18:53:11,260] A new study created in RDB with name: LogisticRegression_meta_first_stat


In [74]:
# ищем лучшую комбинацию гиперпараметров n_trials раз
optuna_study_lg_meta.optimize(optuna_lg_meta, n_trials=300)

[I 2025-04-20 18:53:13,757] Trial 0 finished with values: [0.7299600768685355, 0.7220600195811078, 0.06760945370011623, 1.0, 0.0349874686716792] and parameters: {'C': 0.2550955697667108, 'class_0_weight': 0.014672615376483232, 'class_1_weight': 0.7271518592412085, 'k_features': 233, 'class_1_percent': 0.3697631870543308, 'random_state': 296019}.
[I 2025-04-20 18:53:15,604] Trial 1 finished with values: [0.6763509998100573, 0.6713038886444724, 0.07703684855039043, 0.9253104106972302, 0.040191498718087006] and parameters: {'C': 0.9790529132703446, 'class_0_weight': 0.7844212953776001, 'class_1_weight': 0.8394097630680765, 'k_features': 21, 'class_1_percent': 0.667117363122253, 'random_state': 721723}.
[I 2025-04-20 18:53:19,009] Trial 2 finished with values: [0.7323073910357868, 0.7227571551710509, 0.13702170301416103, 0.4962750716332378, 0.0794835709478064] and parameters: {'C': 0.3257632574308981, 'class_0_weight': 0.5244354950250142, 'class_1_weight': 0.896128430004445, 'k_features': 

##### <span style="color:MediumSlateBlue">Анализ гиперпараметров</span>

In [75]:
# из полученного результат соврмируем Data Frame
optuna_study_lg_pd = optuna_study_lg_meta.trials_dataframe().dropna()
# переименуем столбы
optuna_study_lg_pd.rename(columns={
    'values_0': 'roc_train',
    'values_1': 'roc_valid',
    'values_2': 'f1_score',
    'values_3': 'recall_1',
    'values_4': 'precision_1'
},inplace=True)
# Выделим время потраченное на обучение модели
optuna_study_lg_pd.tail()

,number,roc_train,roc_valid,f1_score,recall_1,precision_1,datetime_start,datetime_complete,duration,params_C,params_class_0_weight,params_class_1_percent,params_class_1_weight,params_k_features,params_random_state,state
295,295,0.738451,0.727707,0.144222,0.158548,0.132271,2025-04-20 19:21:52.633749,2025-04-20 19:22:01.731859,0 days 00:00:09.098110,0.978021,0.366845,0.132793,0.694424,308,471344,COMPLETE
296,296,0.728412,0.719076,0.126981,0.126266,0.127705,2025-04-20 19:22:01.737773,2025-04-20 19:22:07.879444,0 days 00:00:06.141671,0.967208,0.364108,0.138004,0.637043,182,470174,COMPLETE
297,297,0.738185,0.727341,0.121309,0.108309,0.137856,2025-04-20 19:22:07.884179,2025-04-20 19:22:15.867755,0 days 00:00:07.983576,0.982616,0.401983,0.126507,0.690340,296,489837,COMPLETE
298,298,0.734438,0.726052,0.135716,0.140019,0.131669,2025-04-20 19:22:15.873917,2025-04-20 19:22:23.792801,0 days 00:00:07.918884,0.044228,0.379585,0.153891,0.601334,303,437409,COMPLETE
299,299,0.738595,0.728156,0.126694,0.637440,0.070337,2025-04-20 19:22:23.798719,2025-04-20 19:22:31.879695,0 days 00:00:08.080976,0.972004,0.136477,0.167092,0.651310,308,504329,COMPLETE


In [76]:
# покаждем статистику обучения
print('Максимальное значение метрики ROC AUC на валидационном наборе:',round(optuna_study_lg_pd['roc_valid'].max(),3))
print('Среднее значение метрики ROC AUC на валидационном наборе:',round(optuna_study_lg_pd['roc_valid'].mean(),3))
print('Максимальное значение метрики recall_1 на валидационном наборе:',round(optuna_study_lg_pd['recall_1'].max(),3))
print('Среднее значение метрики recall_1 на валидационном наборе:',round(optuna_study_lg_pd['recall_1'].mean(),3))
print('Максимальное значение метрики precision_1 на валидационном наборе:',round(optuna_study_lg_pd['precision_1'].max(),3))
print('Среднее значение метрики precision_1 на валидационном наборе:',round(optuna_study_lg_pd['precision_1'].mean(),3))

Максимальное значение метрики ROC AUC на валидационном наборе: 0.73
Среднее значение метрики ROC AUC на валидационном наборе: 0.725
Максимальное значение метрики recall_1 на валидационном наборе: 1.0
Среднее значение метрики recall_1 на валидационном наборе: 0.462
Максимальное значение метрики precision_1 на валидационном наборе: 0.214
Среднее значение метрики precision_1 на валидационном наборе: 0.091


In [78]:
# построим график важности гиперпарметров
optuna.visualization.plot_param_importances(optuna_study_lg_meta, target_name='Score')

In [80]:
# Построим зависимость метрики precision_1 от гипер параметров
fig = px.scatter(
    data_frame=optuna_study_lg_pd,
    x='precision_1', #ось абсцисс
    y=['roc_train', 'roc_valid', 'recall_1','f1_score'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость precision от гиперпараметров модели', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='Величина метрики precision_1',
    yaxis_title='Величина параметра')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

In [110]:
# Визуализируем метрики полученные при optuna оптимизации
fig = px.scatter(
    data_frame=optuna_study_lg_pd[(optuna_study_lg_pd['roc_valid']>0.9999*optuna_study_lg_pd['roc_valid'].max())],
    x='number', #ось абсцисс
    y=['roc_train','roc_valid','recall_1','precision_1'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость качества модели от trials optuna', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='trials optuna',
    yaxis_title='Величина метрики')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

<span style="color:Blue">

Вывод:  

Их всех выбираем точку $number =127$.   
В этой точке самые оптимальные значения $recall$ и $precision$.  
Посмотрим каким значениям гиперпараметров соотвествует выбранная точка.

In [111]:
# определим номер лучшге варианта
best_optuna_number = 127

# сформируем словарь лучших гипирпарметров RandomForestClassifier
best_param_lr = {
    'C' : round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['params_C'].iloc[0],3),
    'class_weight' : {0:round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['params_class_0_weight'].iloc[0],3),
                      1:round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['params_class_1_weight'].iloc[0],3)},
    }

# создадим перменные
best_class_1_percent = optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['params_class_1_percent'].iloc[0]
best_k_features = optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['params_k_features'].iloc[0]
best_random_state = optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['params_random_state'].iloc[0]
# Выведем принятые наилучшие праметры
print('best C:',best_param_lr['C'])
print('best class 0 weight:',best_param_lr['class_weight'][0])
print('best class 1 weight:',best_param_lr['class_weight'][1])
print('best class 1 percent:',round(best_class_1_percent,3))
print('best k features:',best_k_features)
print('best random state:',best_random_state)
print('time for best train:',round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['duration'].iloc[0].seconds/60),'minutes')

print()
print('ROC AUC на обучающем наборе:', round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['roc_train'].iloc[0],3))
print('ROC AUC на валидационном наборе:', round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['roc_valid'].iloc[0],3))
print('precision класса 1:', round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['precision_1'].iloc[0],3))
print('recall класса 1:', round(optuna_study_lg_pd[optuna_study_lg_pd['number']==best_optuna_number]['recall_1'].iloc[0],3))


best C: 0.912
best class 0 weight: 0.269
best class 1 weight: 0.678
best class 1 percent: 0.194
best k features: 334
best random state: 671463
time for best train: 0 minutes

ROC AUC на обучающем наборе: 0.741
ROC AUC на валидационном наборе: 0.73
precision класса 1: 0.089
recall класса 1: 0.436


##### <span style="color:MediumSlateBlue">Обучение модели с лучшими параметрами</span>

In [112]:
# Загружаем обучающие наборы
MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()
my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

# поготовим данные my_train_pd к работе с функцией class_1_percent_samples
my_train_pd.set_index('id',drop=False,inplace=True)

# с помощью функции class_1_percent_samples зададим долю класса 1
list_c1_percent_id = class_1_percent_samples(my_train_pd,best_class_1_percent,random_state=best_random_state)[:1000000]

# подготовим данные для обучения
MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

# освободим память от "тяжелых" и ненужных файлов
del MX_train_pd, my_train_pd
gc.collect()

# с помощью класса SelectKBest получим список лучших признаков
selector = SelectKBest(f_classif, k=best_k_features)
selector.fit(MX_train_balanced, my_train_balanced)
list_best_features = selector.get_feature_names_out()

MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()


# обучаем модель LogisticRegression с наилучшеми параметрами
logistic_regression = linear_model.LogisticRegression(
        **best_param_lr,
        random_state=42,
        max_iter=10000)
logistic_regression.fit(MX_train_balanced, my_train_balanced)

# удаляем крупные файлы чтобы высвободить память 
del MX_train_balanced, my_train_balanced
gc.collect()

# загружаем тестовые наборы
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()[list_best_features].to_numpy()
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_stat_valid').to_pandas()[list_best_features].to_numpy()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_lr__pred_proba = logistic_regression.predict_proba(MX_train)[:,1]
my_valid_lr_pred_proba = logistic_regression.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_lr_pred = logistic_regression.predict(MX_valid)

# удаляем крупные файлы чтобы высвободить память 
del MX_train, MX_valid
gc.collect()

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_lr__pred_proba),3))
print('ROC AUC на валидационном наборе', round(metrics.roc_auc_score(my_valid, my_valid_lr_pred_proba),3))

print('Основные метрики на валидационом наборе:')
print(metrics.classification_report(my_valid,my_valid_lr_pred,zero_division=0))

# time: 2m 30s

ROC AUC на обучающем наборе 0.741
ROC AUC на валидационном наборе 0.73
Основные метрики на валидационом наборе:
              precision    recall  f1-score   support

           0       0.98      0.84      0.90    144390
           1       0.09      0.44      0.15      5235

    accuracy                           0.82    149625
   macro avg       0.53      0.64      0.52    149625
weighted avg       0.95      0.82      0.87    149625



#### <span style="color:MediumBlue">Построение модели Gradient Boosting Classifier</span>

##### <span style="color:MediumSlateBlue">Baseline обучение модели Hist Gradient Boosting Classifier</span>

In [84]:
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv').set_index('id')['flag']
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_stat_valid').to_pandas()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv').set_index('id')['flag']

In [85]:
# обучаем модель Gradient Boosting Classifier
gradient_boosting = HistGradientBoostingClassifier(random_state=42)
gradient_boosting.fit(MX_train,my_train)

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_rf__pred_proba = gradient_boosting.predict_proba(MX_train)[:,1]
my_valid_rf_pred_proba = gradient_boosting.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_rf_pred = gradient_boosting.predict(MX_valid)

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_rf__pred_proba),3))
print('ROC AUC на валидационном наборе', round(metrics.roc_auc_score(my_valid, my_valid_rf_pred_proba),3))

print('Основные метрики на валидационом наборе:')
print(metrics.classification_report(my_valid,my_valid_rf_pred))

# time: 30s

ROC AUC на обучающем наборе 0.781
ROC AUC на валидационном наборе 0.732
Основные метрики на валидационом наборе:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    144390
           1       0.00      0.00      0.00      5235

    accuracy                           0.96    149625
   macro avg       0.48      0.50      0.49    149625
weighted avg       0.93      0.96      0.95    149625



<span style="color:Blue">

Выводы:

В сравнение с моделью $HistGradientBoostingClassifier$ построенной   
на несбалансированных данных $transform$ $data$ $stat$:
1. Качество модели по метрике $ROC AUC$ не изменилась.
2. Качество модели по метрике $recall_1$ значительно  
улучшилось. Правда, $recall_0$ уменьшилось.
3. Качество модели по метрике $precision_1$ уменьшилось.   
4. На порядок уменьшилось время обучения модели. Это логично,  
велична сбалансированной выборки всего $107000$ $samples$. 

##### <span style="color:MediumSlateBlue">Подбор гиперпараметров модели</span>

In [86]:
# настроим оптимизацию гипер параметров
def optuna_gb_meta(trial):
  # задаем пространства поиска гиперпараметров
  learning_rate = trial.suggest_float('learning_rate',0.01,0.1)
  max_iter = trial.suggest_int('max_iter', 150, 250,step = 1)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 60,step = 1)
  max_depth = trial.suggest_int('max_depth', 1, 10,step = 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 60,step = 1)
  max_features = trial.suggest_float('max_features',0.1,1)
  l2_regularization = trial.suggest_float('l2_regularization',0.01,1)
  class_0_weight = trial.suggest_float('class_0_weight',0.01,1)
  class_1_weight = trial.suggest_float('class_1_weight',0.01,1)
  k_features = trial.suggest_int('k_features', 20, 342,step = 1)
  class_1_percent = trial.suggest_float('class_1_percent',0.01,1)
  random_state = trial.suggest_int('random_state', 1, 1000000)

  # создаем модель
  optuna_gradient_boosting_meta = HistGradientBoostingClassifier(
      learning_rate= learning_rate,
      max_iter = max_iter,
      max_leaf_nodes =max_leaf_nodes,
      max_depth = max_depth,
      min_samples_leaf = min_samples_leaf,
      max_features=max_features,
      l2_regularization = l2_regularization,
      class_weight={0:class_0_weight,1:class_1_weight},
      random_state=random_state)

  # Загружаем обучающие наборы
  MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()
  my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

  # поготовим данные my_train_pd к работе с функцией class_1_percent_samples
  my_train_pd.set_index('id',drop=False,inplace=True)

  # с помощью функции class_1_percent_samples зададим долю класса 1
  list_c1_percent_id = class_1_percent_samples(my_train_pd,class_1_percent,random_state=random_state)[:1000000]

  # подготовим данные для обучения
  MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
  my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

  # освободим память от "тяжелых" и ненужных файлов
  del MX_train_pd, my_train_pd
  gc.collect()
  
  # с помощью класса SelectKBest получим список лучших признаков
  selector = SelectKBest(f_classif, k=k_features)
  selector.fit(MX_train_balanced, my_train_balanced)
  list_best_features = selector.get_feature_names_out()

  MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()

  # я не воспользовался параметром timeout метода optimize потому, что  
  # optimize отставливает поиск параметров, если время обучения 
  # превышает timeout. Мне нужно чтобы поиск параметров продолжился.
  try:
    # для модуля func_time_out упакуем обучение модели в функцию try_func
    def try_func():
            return optuna_gradient_boosting_meta.fit(MX_train_balanced,my_train_balanced)
    # обучаем модель с ограничением по времени 10 минут (600 секунд)
    optuna_gradient_boosting_meta = func_timeout.func_timeout(600, try_func)

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()

    # загружаем тестовые наборы
    MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()[list_best_features].to_numpy()
    MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_stat_valid').to_pandas()[list_best_features].to_numpy()
    my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
    my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()

    # делаем предсказание на обучающем и валидационном наборе
    # cчитаем метрики для класса 1 добавляем их в список
    roc_train = metrics.roc_auc_score(my_train, optuna_gradient_boosting_meta.predict_proba(MX_train)[:,1])
    roc_valid = metrics.roc_auc_score(my_valid, optuna_gradient_boosting_meta.predict_proba(MX_valid)[:,1])
    f1_score = metrics.f1_score(my_valid, optuna_gradient_boosting_meta.predict(MX_valid))
    recall_1 = metrics.recall_score(my_valid, optuna_gradient_boosting_meta.predict(MX_valid),average=None,zero_division=0)[1]
    precision_1 = metrics.precision_score(my_valid, optuna_gradient_boosting_meta.predict(MX_valid),average=None,zero_division=0)[1]

    # удаляем крупные файлы чтобы высвободить память 
    del MX_train, MX_valid, my_train, my_valid
    gc.collect()

  except func_timeout.FunctionTimedOut:
    # удаляем крупные файлы чтобы высвободить память 
    del MX_train_balanced, my_train_balanced
    gc.collect()
    
    # фиксируем пустые значения метрик
    roc_train = 0
    roc_valid = 0
    f1_score = 0
    recall_1 = 0
    precision_1 = 0
    pass

  return roc_train, roc_valid, f1_score, recall_1, precision_1

In [ ]:
# ну случай удаления обучения
# optuna.delete_study(study_name="HistGradientBoostingClassifier_meta_first_stat", storage='sqlite:///optuna_studies.db')

# cоздаем объект исследования
# чтобы модель не переобучалась минимизируем roc_train 
# и максимизируем roc_valid
optuna_study_gb_meta = optuna.create_study(study_name="HistGradientBoostingClassifier_meta_first_stat", 
                               directions=['maximize','maximize','maximize','maximize','maximize'], 
                               sampler=optuna.samplers.TPESampler(),
                               pruner='Hyperband',
                               storage='sqlite:///optuna_studies.db',
                               load_if_exists=True)

[I 2025-04-20 19:23:05,011] A new study created in RDB with name: HistGradientBoostingClassifier_meta_first_stat


In [88]:
# ищем лучшую комбинацию гиперпараметров n_trials раз
optuna_study_gb_meta.optimize(optuna_gb_meta, n_trials=300)

[I 2025-04-20 19:23:09,654] Trial 0 finished with values: [0.756599372235425, 0.7321073980298372, 0.07309545264263263, 0.9839541547277937, 0.037957613629664565] and parameters: {'learning_rate': 0.08570252712745458, 'max_iter': 203, 'max_leaf_nodes': 54, 'max_depth': 4, 'min_samples_leaf': 17, 'max_features': 0.8743782355085334, 'l2_regularization': 0.1864456683984081, 'class_0_weight': 0.08096485560859071, 'class_1_weight': 0.4736053317550731, 'k_features': 281, 'class_1_percent': 0.5442099341657929, 'random_state': 156058}.
[I 2025-04-20 19:23:14,357] Trial 1 finished with values: [0.7439749719086739, 0.7306861477587132, 0.10473232230951945, 0.8212034383954154, 0.05593286494925839] and parameters: {'learning_rate': 0.06097976522072606, 'max_iter': 211, 'max_leaf_nodes': 31, 'max_depth': 2, 'min_samples_leaf': 44, 'max_features': 0.3140067663478741, 'l2_regularization': 0.39850598635500356, 'class_0_weight': 0.3109791816393976, 'class_1_weight': 0.14619588086978336, 'k_features': 340,

##### <span style="color:MediumSlateBlue">Анализ гиперпараметров</span>

In [89]:
# из полученного результат соврмируем Data Frame
optuna_study_gb_meta_pd = optuna_study_gb_meta.trials_dataframe()
# переименуем столбы
optuna_study_gb_meta_pd.rename(columns={
    'values_0': 'roc_train',
    'values_1': 'roc_valid',
    'values_2': 'f1_score',
    'values_3': 'recall_1',
    'values_4': 'precision_1'
},inplace=True)
optuna_study_gb_meta_pd.tail(5)

,number,roc_train,roc_valid,f1_score,recall_1,precision_1,datetime_start,datetime_complete,duration,params_class_0_weight,...,params_k_features,params_l2_regularization,params_learning_rate,params_max_depth,params_max_features,params_max_iter,params_max_leaf_nodes,params_min_samples_leaf,params_random_state,state
295,295,0.782753,0.735005,0.160974,0.311939,0.108476,2025-04-20 20:28:12.856070,2025-04-20 20:28:55.804379,0 days 00:00:42.948309,0.129293,...,342,0.791043,0.019017,9,0.530224,243,58,1,940750,COMPLETE
296,296,0.771059,0.734394,0.155815,0.201146,0.127159,2025-04-20 20:28:55.809659,2025-04-20 20:29:48.277531,0 days 00:00:52.467872,0.137148,...,337,0.778605,0.011525,9,0.519322,242,57,4,917998,COMPLETE
297,297,0.784517,0.735330,0.142373,0.152627,0.133411,2025-04-20 20:29:48.283069,2025-04-20 20:30:39.932995,0 days 00:00:51.649926,0.130064,...,342,0.826667,0.018519,9,0.524345,240,59,3,936463,COMPLETE
298,298,0.785365,0.735699,0.128655,0.117670,0.141903,2025-04-20 20:30:39.938552,2025-04-20 20:31:30.076055,0 days 00:00:50.137503,0.157550,...,335,0.810109,0.020111,9,0.498496,244,56,1,942875,COMPLETE
299,299,0.779582,0.735173,0.156448,0.365807,0.099501,2025-04-20 20:31:30.081065,2025-04-20 20:32:14.829955,0 days 00:00:44.748890,0.117100,...,341,0.779593,0.017133,9,0.554455,239,58,1,914266,COMPLETE


In [90]:
# покаждем статистику обучения
print('Максимальное значение метрики ROC AUC на валидационном наборе:',round(optuna_study_gb_meta_pd['roc_valid'].max(),3))
print('Среднее значение метрики ROC AUC на валидационном наборе:',round(optuna_study_gb_meta_pd['roc_valid'].mean(),3))
print('Максимальное значение метрики f1_score на валидационном наборе:',round(optuna_study_gb_meta_pd['f1_score'].max(),3))
print('Среднее значение метрики f1_score на валидационном наборе:',round(optuna_study_gb_meta_pd['f1_score'].mean(),3))
print('Максимальное значение метрики recall_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['recall_1'].max(),3))
print('Среднее значение метрики recall_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['recall_1'].mean(),3))
print('Максимальное значение метрики precision_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['precision_1'].max(),3))
print('Среднее значение метрики precision_1 на валидационном наборе:',round(optuna_study_gb_meta_pd['precision_1'].mean(),3))

Максимальное значение метрики ROC AUC на валидационном наборе: 0.736
Среднее значение метрики ROC AUC на валидационном наборе: 0.729
Максимальное значение метрики f1_score на валидационном наборе: 0.163
Среднее значение метрики f1_score на валидационном наборе: 0.111
Максимальное значение метрики recall_1 на валидационном наборе: 1.0
Среднее значение метрики recall_1 на валидационном наборе: 0.418
Максимальное значение метрики precision_1 на валидационном наборе: 0.462
Среднее значение метрики precision_1 на валидационном наборе: 0.104


In [91]:
# Максимальное значение метрики ROC AUC на валидационном наборе: 0.736
# Среднее значение метрики ROC AUC на валидационном наборе: 0.73
# Максимальное значение метрики f1_score на валидационном наборе: 0.161
# Среднее значение метрики f1_score на валидационном наборе: 0.108
# Максимальное значение метрики recall_1 на валидационном наборе: 1.0
# Среднее значение метрики recall_1 на валидационном наборе: 0.458
# Максимальное значение метрики precision_1 на валидационном наборе: 0.353
# Среднее значение метрики precision_1 на валидационном наборе: 0.097

In [92]:
# построим график важности гиперпарметров
optuna.visualization.plot_param_importances(optuna_study_gb_meta, target_name='Score')

In [94]:
# Построим зависимость метрики precision_1 от гиперпараметров
fig = px.scatter(
    data_frame=optuna_study_gb_meta_pd,
    x='precision_1', #ось абсцисс
    y=['recall_1','f1_score','roc_train','roc_valid'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость precision класса 1 от гиперпараметров модели', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='Величина метрики precision класса 1',
    yaxis_title='Величина параметра')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

In [114]:
# Построим зависимость метрик качества модели от number

fig = px.scatter(
    data_frame=optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['roc_valid']>0.9999*optuna_study_gb_meta_pd['roc_valid'].max()],
    x='number', #ось абсцисс
    y=['roc_train','roc_valid','recall_1','precision_1'], #ось ординат
)
fig.update_layout(
    title ={
        'text':'Зависимость precision от гиперпараметров модели', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height =800,# Высота рабочей плоскости
    width = 1350, # Ширина рабочей плоскости
    bargap=0.2, # Добавил расстояния между столбами гистограммы
    xaxis_title='Величина метрики precision_1',
    yaxis_title='Величина параметра')
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)

fig.show()

<span style="color:Blue">

Вывод:  

Их всех выбираем точку $number =272$.   
В этой точке оптимальные значения метрик $recall_1$ и $precision_1$, а  
также относительно высокая метрика $ROC AUC$.   
Посмотрим каким значениям гиперпараметров соотвествует выбранная точка.

In [115]:
# определим номер лучшге варианта
best_optuna_number = 272

# сформируем словарь лучших гипирпарметров HistGradientBoostingClassifier
best_param_hgbc = {
    'learning_rate' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_learning_rate'].iloc[0],
    'max_iter' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_iter'].iloc[0],
    'max_leaf_nodes' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_leaf_nodes'].iloc[0],
    'max_depth' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_depth'].iloc[0],
    'min_samples_leaf' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_min_samples_leaf'].iloc[0],
    'max_features' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_max_features'].iloc[0],
    'l2_regularization' : optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_l2_regularization'].iloc[0],
    'class_weight' : {0: optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_class_0_weight'].iloc[0],
                      1: optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_class_1_weight'].iloc[0],}
    }

# определим перменные для лучших значений параметров
best_k_features = optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_k_features'].iloc[0]
best_random_state = optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_random_state'].iloc[0]
best_class_1_percent = optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['params_class_1_percent'].iloc[0]


# Выведем принятые наилучшие параметры
print('best learning rate:',round(best_param_hgbc['learning_rate'],3))
print('best max iter:',best_param_hgbc['max_iter'])
print('best max leaf nodes:',best_param_hgbc['max_leaf_nodes'])
print('best max depth:',best_param_hgbc['max_depth'])
print('best min samples leaf:',best_param_hgbc['min_samples_leaf'])
print('best max features:',round(best_param_hgbc['max_features'],3))
print('best l2 regularization:',round(best_param_hgbc['l2_regularization'],3))
print('best class 0 weight:',round(best_param_hgbc['class_weight'][0],3))
print('best class 1 weight:',round(best_param_hgbc['class_weight'][1],3))

print('best class 1 percent:',round(best_class_1_percent,3))
print('best k features:',best_k_features)
print('best random state:',best_random_state)
print('time for best train:',round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['duration'].iloc[0].seconds/60),'minutes')

print()
print('ROC AUC на обучающем наборе:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['roc_train'].iloc[0],3))
print('ROC AUC на валидационном наборе:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['roc_valid'].iloc[0],3))
print('precision класса 1:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['precision_1'].iloc[0],3))
print('recall класса 1:', round(optuna_study_gb_meta_pd[optuna_study_gb_meta_pd['number']==best_optuna_number]['recall_1'].iloc[0],3))

best learning rate: 0.025
best max iter: 232
best max leaf nodes: 58
best max depth: 9
best min samples leaf: 7
best max features: 0.346
best l2 regularization: 0.688
best class 0 weight: 0.233
best class 1 weight: 0.616
best class 1 percent: 0.15
best k features: 334
best random state: 975267
time for best train: 0 minutes

ROC AUC на обучающем наборе: 0.788
ROC AUC на валидационном наборе: 0.736
precision класса 1: 0.103
recall класса 1: 0.333


##### <span style="color:MediumSlateBlue">Обучение модели с лучшими параметрами</span>

In [116]:
# Загружаем обучающие наборы
MX_train_pd = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()
my_train_pd = pd.read_csv('features/firstmeta/mfy_train.csv')

# поготовим данные my_train_pd к работе с функцией class_1_percent_samples
my_train_pd.set_index('id',drop=False,inplace=True)

# с помощью функции class_1_percent_samples зададим долю класса 1
list_c1_percent_id = class_1_percent_samples(my_train_pd,best_class_1_percent,random_state=best_random_state)[:1000000]

# подготовим данные для обучения
MX_train_balanced = MX_train_pd.loc[list_c1_percent_id]
my_train_balanced = my_train_pd.loc[list_c1_percent_id]['flag'].to_numpy()

# освободим память от "тяжелых" и ненужных файлов
del MX_train_pd, my_train_pd
gc.collect()

# с помощью класса SelectKBest получим список лучших признаков
selector = SelectKBest(f_classif, k=best_k_features)
selector.fit(MX_train_balanced, my_train_balanced)
list_best_features = selector.get_feature_names_out()

MX_train_balanced = MX_train_balanced[list_best_features].to_numpy()

# обучаем модель HistGradientBoostingClassifier с наилучшеми параметрами
gradient_boosting = HistGradientBoostingClassifier(
        **best_param_hgbc,
        random_state=best_random_state)
gradient_boosting.fit(MX_train_balanced,my_train_balanced)

# удаляем крупные файлы чтобы высвободить память 
del MX_train_balanced, my_train_balanced
gc.collect()

# загружаем тестовые наборы
MX_train = fp.ParquetFile('features/firstmeta/features_first_meta_stat_train').to_pandas()[list_best_features].to_numpy()
MX_valid = fp.ParquetFile('features/firstmeta/features_first_meta_stat_valid').to_pandas()[list_best_features].to_numpy()
my_train = pd.read_csv('features/firstmeta/mfy_train.csv')['flag'].to_numpy()
my_valid = pd.read_csv('features/firstmeta/mfy_valid.csv')['flag'].to_numpy()

# для метрик ROC AUC делаем предсказание модели в виде вероятности
my_train_gb_pred_proba = gradient_boosting.predict_proba(MX_train)[:,1]
my_valid_gb_pred_proba = gradient_boosting.predict_proba(MX_valid)[:,1]

# Делаем предсказание для валидационной выборки
my_valid_gb_pred = gradient_boosting.predict(MX_valid)

# удаляем крупные файлы чтобы высвободить память 
del MX_train, MX_valid
gc.collect()

#Выводим значения метрик
print('ROC AUC на обучающем наборе', round(metrics.roc_auc_score(my_train, my_train_gb_pred_proba),3))
print('ROC AUC на валидационном наборе', round(metrics.roc_auc_score(my_valid, my_valid_gb_pred_proba),3))

print('Основные метрики на валидационом наборе:')
print(metrics.classification_report(my_valid,my_valid_gb_pred,zero_division=0))

# time: 1m 40s

ROC AUC на обучающем наборе 0.788
ROC AUC на валидационном наборе 0.736
Основные метрики на валидационом наборе:
              precision    recall  f1-score   support

           0       0.97      0.90      0.93    144390
           1       0.10      0.33      0.16      5235

    accuracy                           0.88    149625
   macro avg       0.54      0.61      0.55    149625
weighted avg       0.94      0.88      0.91    149625

